# 1*. Cài Đặt Cấu Hình

In [428]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.decomposition import PCA
from scipy.fft import fft, fftfreq
import pywt

# 2 Phân Tách Dữ Liệu Thế Giới

## 2.0* Function

In [429]:
def print_dictionary_table(dictionary, size=7):
    keys = list(dictionary.keys())
    keys = [' '*(size-len(str(e))) + str(e) for e in keys]
    values = list(dictionary.values())
    values = [' '*(size-len(str(e))) + str(e) for e in values]
    table =  [keys, values]
    print(tabulate(table, headers="firstrow", tablefmt="grid"))

def check_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values_dict = {col: int(count) for col, count in missing_values.items() if count > 0}
    empty_strings = (df == "").sum()
    empty_string_columns_dict = {col: int(count) for col, count in empty_strings.items() if count > 0}
    total_rows = df.shape[0]
    missing_percent_dict = {col: f"{round((count / total_rows) * 100, 3)}%" for col, count in missing_values.items() if count > 0}
    result = {
        "missing_values": missing_values_dict,
        "empty_string_columns": empty_string_columns_dict,
        "missing_percent": missing_percent_dict
    }

    return result

def check_dtypes(df):
    dtypes_dict = df.dtypes.apply(lambda x: x.name).to_dict()
    return dtypes_dict

def count_duplicate_rows(df, id_column):
    columns_to_check = [col for col in df.columns if col != id_column]
    count = df.duplicated(subset=columns_to_check, keep=False).sum()
    return count

def count_unique_values(data):
    unique_counts = {}
    for column in data.columns:
        unique_counts[column] = data[column].nunique()
    return unique_counts

def print_unique_values_stats(df, column_name):
    if column_name not in df.columns:
        print(f"Cột '{column_name}' không tồn tại trong DataFrame.")
        return
    value_counts = df[column_name].value_counts()
    total = len(df)
    stats_df = pd.DataFrame({
        'Giá trị cột ' + column_name: value_counts.index,
        'Số lượng': value_counts.values,
        'Phần trăm (%)': (value_counts.values / total) * 100
    })
    stats_df = stats_df.sort_values(by='Số lượng', ascending=False)
    rows = stats_df.to_dict(orient='records')
    print(tabulate(rows, headers="keys", tablefmt="grid"))

def khai_quat_du_lieu(df):
  print("- Kích thước của dữ liệu:", df.shape)

  print("\n- Tên các cột:", df.columns.tolist())

  print("\n- Kiểu dữ liệu của các cột: ")
  print_dictionary_table(check_dtypes(df), 5)

  print("\n- Kiểm tra các cột thiếu dữ liệu")
  d_miss = check_missing_values(df)
  print(" + Thiếu giá trị  : ", len(d_miss['missing_values']), " cột")
  print_dictionary_table(d_miss['missing_values'], 5)
  print(" + Tính phần trăm : ", len(d_miss['missing_percent']), "cột")
  print_dictionary_table(d_miss['missing_percent'], 5)
  print(" + Chuỗi rỗng     : ", d_miss['empty_string_columns'])

  print("\n- Số lượng các dòng trùng lặp (không tính object_id): ", count_duplicate_rows(df, 0))

  print("\n- Số lượng các giá trị khác nhau trong các cột")
  print_dictionary_table(count_unique_values(df), 5)


def get_value_by_id_column(df, id_column_name, id_value, value_column_name):
    try:
        row_index = df[df[id_column_name] == id_value].index[0]
        value = df.loc[row_index, value_column_name]
        if pd.isna(value):  # Kiểm tra nếu giá trị là NaN
            return None
        return value
    except IndexError:  # Xử lý trường hợp không tìm thấy id_value
        return None
    except KeyError:  # Xử lý trường hợp cột không tồn tại
        return None

def in_vi_tri_gia_tri_thieu(dataframe, id_col):
    cot_gia_tri_thieu = dataframe.columns[dataframe.isnull().any()]
    thong_tin_gia_tri_thieu = []
    for col in cot_gia_tri_thieu:
        for index, value in dataframe[col].items():
            if pd.isnull(value):
                thong_tin_gia_tri_thieu.append({"ID": dataframe.at[index, id_col], "Tên Cột": col})
    df_thong_tin = pd.DataFrame(thong_tin_gia_tri_thieu)
    print(tabulate(df_thong_tin, headers='keys', tablefmt='grid'))


def in_tuong_quan(correlation):
  corr_df = correlation.to_frame()
  corr_df_flipped = corr_df.transpose()
  print(tabulate(corr_df_flipped, headers='keys', tablefmt='psql'))

def thong_ke_mo_ta_1_hang(df, row_index=-1, col_name = None):
  if row_index == -1:
    row_data = df[col_name]
  else:
    row_data = df.iloc[row_index]
  return {
        'mean': row_data.mean(),
        'std': row_data.std(),
        'min': row_data.min(),
        '25%': row_data.quantile(0.25),
        '50% (median)': row_data.median(),
        '75%': row_data.quantile(0.75),
        'max': row_data.max(),
        'sum': row_data.sum(),
        'count': row_data.count(),
        'variance': row_data.var(),
        'skewness': row_data.skew(),
        'kurtosis': row_data.kurtosis()}

def in_bang_thong_ke_mo_ta(df, row_index=-1, col_name = None):
  statistics = thong_ke_mo_ta_1_hang(df, row_index, col_name)
  headers = list(statistics.keys())
  values = list(statistics.values())
  table = [values]
  print(tabulate(table, headers=headers, tablefmt='grid'))


def ve_bieu_do_thong_ke_mo_ta(df, transpose=False):
    if transpose is True:
        df = df.transpose()
    fig = go.Figure()
    swarm_data = []
    box_data = []
    for i, col in enumerate(df.columns):
        swarm_data.append(go.Scatter(x=[col] * len(df[col]), y=df[col], mode='markers', name=col, marker=dict(size=6, opacity=0.6)))
        box_data.append(go.Box(y=df[col], name=col, boxmean='sd', marker=dict(size=6, opacity=0.6)))
        quantiles = stats.probplot(df[col], dist="norm", fit=False)
        x_values = quantiles[0]
        y_values = quantiles[1]
        qq_line = go.Scatter(x=x_values, y=y_values, mode='lines', name='Q-Q Plot', line=dict(color='red', dash='dash'))
        fig.add_trace(qq_line)
    fig.add_traces(swarm_data + box_data)
    fig.update_layout(title='BoxPlot And QQ Line', xaxis=dict(title='Variable'), yaxis=dict(title='Value'), plot_bgcolor='white')
    fig.show()


def chuan_hoa_min_max(row, min_val=None, max_val=None):
  if min_val is None:
      min_val = np.min(row)
  if max_val is None:
      max_val = np.max(row)
  return [(row - min_val) / (max_val - min_val)]


def ve_bieu_do_tuong_quan(dataframe, title='Correlation Heatmap'):
    fig = go.Figure(data=go.Heatmap(z=dataframe.corr().values,
                                    x=dataframe.columns,
                                    y=dataframe.columns,
                                    colorscale='Spectral'))
    for i in range(len(dataframe.columns)):
        for j in range(len(dataframe.columns)):
            fig.add_annotation(x=dataframe.columns[i],
                               y=dataframe.columns[j],
                               text=str(round(dataframe.corr().iloc[i, j], 2)),
                               font=dict(color='white'),
                               showarrow=False)
    fig.update_layout(title=title)
    fig.show()


def ve_bieu_do_so_sanh_fft(signal1, signal2, fs=1000, name1='Signal 1 FFT', name2='Signal 2 FFT'):
    """
    Vẽ so sánh giữa hai tín hiệu bằng phương pháp Biến đổi Fourier và FFT bằng Plotly.

    Tham số:
    - signal1 (array): Tín hiệu mẫu 1.
    - signal2 (array): Tín hiệu mẫu 2.
    - fs (int, optional): Tần số lấy mẫu (Hz). Mặc định là 1000 Hz.
    """
    # Biến đổi Fourier của hai tín hiệu
    freqs1 = np.fft.fftfreq(len(signal1), 1/fs)
    fft1 = np.fft.fft(signal1)
    freqs2 = np.fft.fftfreq(len(signal2), 1/fs)
    fft2 = np.fft.fft(signal2)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=freqs1, y=np.abs(fft1), mode='lines', name=name1))
    fig.add_trace(go.Scatter(x=freqs2, y=np.abs(fft2), mode='lines', name=name2))

    fig.update_layout(xaxis_title='Frequency (Hz)',
                      yaxis_title='Magnitude',
                      title='FFT Comparison between Signal 1 and Signal 2',
                      plot_bgcolor='rgba(255, 255, 255, 1)',
                      paper_bgcolor='rgba(255, 255, 255, 1)')

    fig.show()

def phantich_pca(data):
    pivot_data = data.pivot(index='Year', columns='ISO3', values='Value').fillna(0)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(pivot_data)
    pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(2)])
    pca_df['Year'] = pivot_data.index
    pca_weights = pd.DataFrame(pca.components_, columns=pivot_data.columns, index=[f'PC{i+1}' for i in range(2)])
    print("PCA Weight:")
    print(pca_weights)
    fig = px.scatter(pca_df, x='PC1', y='PC2', color='Year', template='plotly_white')
    fig.show()


def tao_du_lieu_pca(df, time_line, danh_sach_ma_iso):
  new_time_line = time_line * len(danh_sach_ma_iso)
  new_iso = [item for item in danh_sach_ma_iso for _ in range(len(time_line))]
  np_mat = np.full((len(new_time_line), 1), np.nan)
  for i, row in enumerate(new_time_line):
    #np_mat[i, 0] = int(row)
    #np_mat[i, 2] = new_iso[i]
    np_mat[i, 0] = get_value_by_id_column(df, 'ISO3', new_iso[i], row)
  new_df = pd.DataFrame(np_mat, columns=['Value'], dtype=np.double)
  new_df.insert(0, 'Year', new_time_line)
  new_df.insert(1, 'ISO3', new_iso)
  return new_df


def ve_do_thi_bien_doi_fourier(x, fs, time_labels=None):
    n = len(x)
    fourier = np.fft.fft(x)
    freqs = np.fft.fftfreq(n, 1/fs)
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=np.arange(len(x))/fs, y=x, mode='lines', name='Original Signal'))
    fig1.update_layout(title='Original Signal', xaxis_title='Time (s)', yaxis_title='Amplitude', plot_bgcolor='white')

    if time_labels is not None:
        fig1.update_layout(xaxis=dict(tickmode='array', tickvals=np.arange(len(x))/fs, ticktext=time_labels))

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=freqs, y=np.abs(fourier), mode='lines', name='Magnitude'))
    fig2.update_layout(title='Fourier Transform', xaxis_title='Frequency (Hz)', yaxis_title='Magnitude', plot_bgcolor='white')

    fig1.show()
    fig2.show()

    print("Fourier Coefficients:")
    for i, coeff in enumerate(fourier):
        print(f"Frequency: {freqs[i]} Hz, Magnitude: {np.abs(coeff)}")

## 2.1* Đọc các tập dữ liệu

In [ ]:
d_names = {
    'NhietDo': 'Indicator_3_1_Climate_Indicators_Annual_Mean_Global_Surface_Temperature_577579683071085080.csv',
    'MucNuoc': 'Indicator_3_3_melted_new_950412061812209490.csv',
    'ThienTai': 'Indicator_11_1_Physical_Risks_Climate_related_disasters_frequency_7212563912390016675.csv',
    'RungCarbon': 'Forest_and_Carbon.csv'
}

d_data, d_exdata = {}, {}

for k, v in d_names.items():
  d_data.update({k: pd.read_csv(v, index_col=0)})

# Lấy tất cả các mã ISO3 được sử dụng trong tập dữ liệu
def get_all_ISO3_in_data():
  sum = (d_data['NhietDo']['ISO3'].unique()).tolist()
  sum += (d_data['RungCarbon']['ISO3'].unique()).tolist()
  sum += (d_data['ThienTai']['ISO3'].unique()).tolist()
  return sorted(list(set(sum)))


ISO3 = get_all_ISO3_in_data()
print(ISO3)


## 2.2 Khai mở dữ liệu

### 2.2.1 Khai mở tập dữ liệu Annual Surface Temperature Change

In [ ]:
d_data['NhietDo'].head(2)

In [ ]:
khai_quat_du_lieu(d_data['NhietDo'])

***Nhận Xét***

**Ý nghĩa các cột dữ liệu:**

+ **Country**: Tên của quốc gia.
+ **ISO2**: Đây là mã định danh của quốc gia được viết dưới dạng hai ký tự chữ cái viết hoa, ví dụ: "US" cho Hoa Kỳ, "GB" cho Vương quốc Anh. Mã ISO 2 thường được sử dụng trong các ứng dụng và hệ thống mà yêu cầu một mã định danh ngắn gọn cho quốc gia.

+ **ISO3**: Đây là mã định danh của quốc gia được viết dưới dạng ba ký tự chữ cái viết hoa, ví dụ: "USA" cho Hoa Kỳ, "GBR" cho Vương quốc Anh. Mã ISO 3 cung cấp một cách định danh duy nhất hơn cho các quốc gia, đặc biệt hữu ích khi cần phân biệt giữa các quốc gia có cùng mã ISO 2 nhưng khác nhau về lãnh thổ, ví dụ: "GB" (Vương quốc Anh) và "GB" (Quần đảo Guernsey).

+ **Indicator**: Temperature change with respect to a baseline climatology, corresponding to the period 1951-1980" là một chỉ số thống kê về sự thay đổi nhiệt độ so với một tiêu chuẩn khí hậu cơ bản, tương ứng với giai đoạn từ 1951 đến 1980. Bằng cách so sánh nhiệt độ hiện tại với nhiệt độ trung bình trong giai đoạn thời gian này, chúng ta có thể đánh giá được mức độ thay đổi nhiệt độ trong quá khứ và hiện tại, từ đó có thể phân tích được xu hướng biến đổi khí hậu.

+ **Unit**: Degree Celsius  (Đơn vị là Độ Celsius) chỉ đơn vị đo nhiệt độ được sử dụng trong đoạn mô tả về chỉ số thống kê về sự thay đổi nhiệt độ. Đơn vị này cho biết rằng dữ liệu về nhiệt độ được biểu diễn bằng độ Celsius (°C).

+ **Source**: Food and Agriculture Organization of the United Nations (FAO). 2022. FAOSTAT Climate Change, Climate Indicators, Temperature change. License: CC BY-NC-SA 3.0 IGO. Extracted from: https://www.fao.org/faostat/en/#data/ET. Accessed on 2023-03-28".
+ **CTS Code**: ECCS là mã code của chỉ số từ hệ thống CTS (Contemporary Trends and Structures).
+ **CTS Name**: Surface Temperature Change là tên viết tắt của chỉ số từ hệ thống CTS.
+ **CTS Full Descriptor**: Environment, Climate Change, Climate and Weather, Surface Temperature Change là Mô tả đầy đủ của chỉ số từ hệ thống CTS.
+ **1961-2023**: Giá trị nhiệt độ theo từng năm

***Kết luận: Sử dụng cột IOS3 và các cột năm từ 1961-2023 làm dữ liệu về nhiệt độ của các quốc gia, các cột khác không sử dụng để tính toán.***



### 2.2.2 Khai mở tập dữ liệu Change in Mean Sea Levels

In [296]:
d_data['MucNuoc'].head(2)

,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,Measure,Date,Value
ObjectId,,,,,,,,,,,,
1,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weath...",Sea Okhotsk,1992-12-16,10.11
2,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate and Weath...",Adriatic Sea,1992-12-17,32.73


In [297]:
khai_quat_du_lieu(d_data['MucNuoc'])

- Kích thước của dữ liệu: (39607, 12)

- Tên các cột: ['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS Code', 'CTS Name', 'CTS Full Descriptor', 'Measure', 'Date', 'Value']

- Kiểu dữ liệu của các cột: 
+-----------+---------+---------+-------------+---------+----------+------------+------------+-----------------------+-----------+----------------+---------+
| Country   |  ISO2   |  ISO3   | Indicator   |  Unit   | Source   | CTS Code   | CTS Name   | CTS Full Descriptor   | Measure   |  Date          | Value   |
+===========+=========+=========+=============+=========+==========+============+============+=======================+===========+================+=========+
| object    | float64 | object  | object      | object  | object   | object     | object     | object                | object    | datetime64[ns] | float64 |
+-----------+---------+---------+-------------+---------+----------+------------+------------+-----------------------+-----------+----------------+---

In [298]:
print_unique_values_stats(d_data['MucNuoc'], 'Indicator')
print_unique_values_stats(d_data['MucNuoc'], 'Measure')

+-----------------------------------------------------+------------+-----------------+
| Giá trị cột Indicator                               |   Số lượng |   Phần trăm (%) |
+=====================================================+============+=================+
| Change in mean sea level: Sea level: TOPEX.Poseidon |      10946 |      27.6365    |
+-----------------------------------------------------+------------+-----------------+
| Change in mean sea level: Sea level: Jason.1        |      10348 |      26.1267    |
+-----------------------------------------------------+------------+-----------------+
| Change in mean sea level: Sea level: Jason.2        |       8169 |      20.6251    |
+-----------------------------------------------------+------------+-----------------+
| Change in mean sea level: Sea level: Jason.3        |       7361 |      18.5851    |
+-----------------------------------------------------+------------+-----------------+
| Change in mean sea level: Sea level: Sent

***Nhận Xét***

**Ý nghĩa các cột dữ liệu:**

+ **Country**: Tên của quốc gia.
+ **ISO2**: Đây là mã định danh của quốc gia được viết dưới dạng hai ký tự chữ cái viết hoa, ví dụ: "US" cho Hoa Kỳ, "GB" cho Vương quốc Anh. Mã ISO 2 thường được sử dụng trong các ứng dụng và hệ thống mà yêu cầu một mã định danh ngắn gọn cho quốc gia.

+ **ISO3**: Đây là mã định danh của quốc gia được viết dưới dạng ba ký tự chữ cái viết hoa, ví dụ: "USA" cho Hoa Kỳ, "GBR" cho Vương quốc Anh. Mã ISO 3 cung cấp một cách định danh duy nhất hơn cho các quốc gia, đặc biệt hữu ích khi cần phân biệt giữa các quốc gia có cùng mã ISO 2 nhưng khác nhau về lãnh thổ, ví dụ: "GB" (Vương quốc Anh) và "GB" (Quần đảo Guernsey).

+ **Indicator**: có là giá trị "Change in mean sea level: Sea level: TOPEX.Poseidon" cho biết sự thay đổi trong mực nước biển trung bình được đo bằng vệ tinh TOPEX/Poseidon. Dữ liệu này giúp theo dõi biến động mực nước biển trên toàn cầu theo thời gian, từ đó cung cấp thông tin quan trọng cho nghiên cứu và quản lý môi trường biển. Ngoài ra còn có 5 vệ tinh khác như Jason.1, Jason.2, Json3, Sentinel-6MF, Trend.

+ **Unit**: Millimeters (mm): Đơn vị đo lường chiều dài trong hệ mét, tương đương với một phần nghìn mét (1 mm = 0.001 mét). Ví dụ, nếu giá trị là 5 mm, điều này có nghĩa là mực nước biển đã thay đổi (tăng hoặc giảm) 5 milimet so với một mức chuẩn hoặc thời điểm trước đó.

+ **Source**: có giá trị "National Oceanic and Atmospheric Administration (NOAA). 2020. Sea Level Rise. Laboratory for Satellite Altimetry. https://www.star.nesdis.noaa.gov/socd/lsa/SeaLevelRise/LSA_SLR_timeseries.php. Accessed on 2022-12-08." .
+ **CTS Code**: ECCL là mã code của chỉ số từ hệ thống CTS (Contemporary Trends and Structures).
+ **CTS Name**: Change in Mean Sea Level là tên viết tắt của chỉ số từ hệ thống CTS.
+ **CTS Full Descriptor**: Environment, Climate Change, Climate and Weather, Change in Mean Sea Level là Mô tả đầy đủ của chỉ số từ hệ thống CTS.
+ **Measure**: dùng để xác định rõ ràng khu vực địa lý hoặc vùng biển mà dữ liệu mực nước biển được thu thập, có 25 giá trị theo từng khu vực địa lý
+ **Date**: Thời gian đo lường mực nước
+ **Value**: Giá trị mực nước biển

***Kết luận: Sử dụng cột Measure, Date và Value làm dữ liệu về mực nước biển của các quốc gia, các cột khác không sử dụng để tính toán.***



### 2.2.3 Khai mở tập dữ liệu Forest and Carbon

In [299]:
d_data['RungCarbon'].head(2)

,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,F1992,...,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020
ObjectId,,,,,,,,,,,,,,,,,,,,,
1,Advanced Economies,NaN,AETMP,Carbon stocks in forests,Million tonnes,Food and Agriculture Organization of the Unite...,NaN,NaN,NaN,52618.6601,...,56445.5501,56668.6489,56891.748,57114.8466,57337.9461,57568.1744,57777.8499,57959.7625,58035.0513,58102.5521
2,Advanced Economies,NaN,AETMP,Forest area,1000 HA,Food and Agriculture Organization of the Unite...,NaN,NaN,NaN,945840.6089,...,956338.5036,957498.4138,958658.324,959818.2342,960978.1444,962013.8455,961845.1200,961963.7100,962100.5900,962252.1900


In [300]:
khai_quat_du_lieu(d_data['RungCarbon'])

- Kích thước của dữ liệu: (1353, 38)

- Tên các cột: ['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor', 'F1992', 'F1993', 'F1994', 'F1995', 'F1996', 'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020']

- Kiểu dữ liệu của các cột: 
+-----------+---------+---------+-------------+---------+----------+------------+------------+-----------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Country   |  ISO2   |  ISO3   | Indicator   |  Unit   | Source   | CTS_Code   | CTS_Name   | CTS_Full_Descriptor   | F1992   | F1993   | F1

In [301]:
print_unique_values_stats(d_data['RungCarbon'], 'Indicator')
print_unique_values_stats(d_data['RungCarbon'], 'Unit')

+-----------------------------------+------------+-----------------+
| Giá trị cột Indicator             |   Số lượng |   Phần trăm (%) |
+===================================+============+=================+
| Forest area                       |        235 |         17.3688 |
+-----------------------------------+------------+-----------------+
| Land area                         |        235 |         17.3688 |
+-----------------------------------+------------+-----------------+
| Share of forest area              |        235 |         17.3688 |
+-----------------------------------+------------+-----------------+
| Index of forest extent            |        234 |         17.2949 |
+-----------------------------------+------------+-----------------+
| Carbon stocks in forests          |        208 |         15.3732 |
+-----------------------------------+------------+-----------------+
| Index of carbon stocks in forests |        206 |         15.2254 |
+---------------------------------

***Nhận Xét***

**Ý nghĩa các cột dữ liệu:**

+ **Country**: Tên của quốc gia.
+ **ISO2**: Đây là mã định danh của quốc gia được viết dưới dạng hai ký tự chữ cái viết hoa, ví dụ: "US" cho Hoa Kỳ, "GB" cho Vương quốc Anh. Mã ISO 2 thường được sử dụng trong các ứng dụng và hệ thống mà yêu cầu một mã định danh ngắn gọn cho quốc gia.

+ **ISO3**: Đây là mã định danh của quốc gia được viết dưới dạng ba ký tự chữ cái viết hoa, ví dụ: "USA" cho Hoa Kỳ, "GBR" cho Vương quốc Anh. Mã ISO 3 cung cấp một cách định danh duy nhất hơn cho các quốc gia, đặc biệt hữu ích khi cần phân biệt giữa các quốc gia có cùng mã ISO 2 nhưng khác nhau về lãnh thổ, ví dụ: "GB" (Vương quốc Anh) và "GB" (Quần đảo Guernsey).

+ **Indicator**: chứa các chỉ số (indicator) liên quan đến diện tích rừng và lượng carbon. Các chỉ số cụ thể bao gồm:
        Forest area: Diện tích rừng (số lượng: 235, chiếm 17.3688%).
        Land area: Diện tích đất (số lượng: 235, chiếm 17.3688%).
        Share of forest area: Tỷ lệ diện tích rừng (số lượng: 235, chiếm 17.3688%).
        Index of forest extent: Chỉ số mở rộng diện tích rừng (số lượng: 234, chiếm 17.2949%).
        Carbon stocks in forests: Lượng carbon tích trữ trong rừng (số lượng: 208, chiếm 15.3732%).
        Index of carbon stocks in forests: Chỉ số lượng carbon tích trữ trong rừng (số lượng: 206, chiếm 15.2254%).

+ **Unit**: chứa các đơn vị đo lường tương ứng với các chỉ số trong cột "Indicator". Các đơn vị đo lường cụ thể bao gồm:
        1000 HA: Đơn vị nghìn hecta, dùng để đo diện tích rừng và đất (số lượng: 470, chiếm 34.7376%).
        Index: Chỉ số, thường dùng để đo lường mức độ hay thay đổi theo thời gian (số lượng: 440, chiếm 32.5203%).
        Percent: Phần trăm, thường dùng để đo tỷ lệ diện tích rừng so với diện tích đất tổng cộng (số lượng: 235, chiếm 17.3688%).
        Million tonnes: Đơn vị triệu tấn, dùng để đo lường lượng carbon tích trữ trong rừng (số lượng: 208, chiếm 15.3732%).

+ **Source**: có giá trị Food and Agriculture Organization of the United Nations (FAO), 2022. FAO, 2022 FAOSTAT Land, Inputs and Sustainability, Land Use http://www.fao.org/faostat/en/#data/RL, Rome, FAO. Accessed on: 03-28-2023; IMF staff calculations .
+ **CTS Code**: Không có.
+ **CTS Name**: Không có.
+ **CTS Full Descriptor**: Không có.
+ **F192-F2020**: Giá trị

***Kết luận: Sử dụng cột ISO3, Indicator, Unit và các cột F1992-F2020 làm dữ liệu về rừng và carbon của các quốc gia, các cột khác không sử dụng để tính toán.***



### 2.2.4 Khai mở tập dữ liệu Climate-related Disasters Frequency

In [302]:
d_data['ThienTai'].head(2)

,Country,ISO2,ISO3,Indicator,Unit,Source,CTS Code,CTS Name,CTS Full Descriptor,1980,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
ObjectId,,,,,,,,,,,,,,,,,,,,,
1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
khai_quat_du_lieu(d_data['ThienTai'])

- Kích thước của dữ liệu: (970, 52)

- Tên các cột: ['Country', 'ISO2', 'ISO3', 'Indicator', 'Unit', 'Source', 'CTS Code', 'CTS Name', 'CTS Full Descriptor', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+-----------+---------+---------+-------------+---------+----------+------------+------------+-----------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

In [304]:
print_unique_values_stats(d_data['ThienTai'], 'Indicator')

+-------------------------------------------------------------------------------+------------+-----------------+
| Giá trị cột Indicator                                                         |   Số lượng |   Phần trăm (%) |
+===============================================================================+============+=================+
| Climate related disasters frequency, Number of Disasters: TOTAL               |        215 |        22.1649  |
+-------------------------------------------------------------------------------+------------+-----------------+
| Climate related disasters frequency, Number of Disasters: Storm               |        188 |        19.3814  |
+-------------------------------------------------------------------------------+------------+-----------------+
| Climate related disasters frequency, Number of Disasters: Flood               |        182 |        18.7629  |
+-------------------------------------------------------------------------------+------------+--

***Nhận Xét***

**Ý nghĩa các cột dữ liệu:**

+ **Country**: Tên của quốc gia.
+ **ISO2**: Đây là mã định danh của quốc gia được viết dưới dạng hai ký tự chữ cái viết hoa, ví dụ: "US" cho Hoa Kỳ, "GB" cho Vương quốc Anh. Mã ISO 2 thường được sử dụng trong các ứng dụng và hệ thống mà yêu cầu một mã định danh ngắn gọn cho quốc gia.

+ **ISO3**: Đây là mã định danh của quốc gia được viết dưới dạng ba ký tự chữ cái viết hoa, ví dụ: "USA" cho Hoa Kỳ, "GBR" cho Vương quốc Anh. Mã ISO 3 cung cấp một cách định danh duy nhất hơn cho các quốc gia, đặc biệt hữu ích khi cần phân biệt giữa các quốc gia có cùng mã ISO 2 nhưng khác nhau về lãnh thổ, ví dụ: "GB" (Vương quốc Anh) và "GB" (Quần đảo Guernsey).

+ **Indicator**: mô tả loại hình và tần suất của các thảm họa liên quan đến khí hậu mà mỗi quốc gia đã trải qua. Dưới đây là chi tiết về các giá trị có thể có trong cột này và ý nghĩa của chúng:
        Climate related disasters frequency, Number of Disasters: TOTAL - Tổng số thảm họa liên quan đến khí hậu.
        Climate related disasters frequency, Number of Disasters: Storm - Số lần xảy ra các cơn bão, bao gồm bão nhiệt đới, lốc xoáy và các hiện tượng thời tiết bão khác.
        Climate related disasters frequency, Number of Disasters: Flood - Số lần xảy ra các trận lụt, bao gồm lụt sông, lụt bờ biển và các hiện tượng lụt khác.
        Climate related disasters frequency, Number of Disasters: Landslide - Số lần xảy ra lở đất, sạt lở đất do mưa lớn, địa chấn hoặc các nguyên nhân khác.
        Climate related disasters frequency, Number of Disasters: Drought - Hạn hán
        Climate related disasters frequency, Number of Disasters: Wildfire - Số lần xảy ra cháy rừng, bao gồm các vụ cháy do thiên nhiên hoặc con người gây ra.
        Climate related disasters frequency, Number of Disasters: Extreme temperature - Số lần xảy ra hiện tượng nhiệt độ cực đoan, bao gồm các đợt nắng nóng, lạnh cực đoan.

+ **Unit**: Number of - là số lần xảy ra các thảm họa.

+ **Source**: cThe Emergency Events Database (EM-DAT) , Centre for Research on the Epidemiology of Disasters (CRED) / Université catholique de Louvain (UCLouvain), Brussels, Belgium – www.emdat.be. Accessed on 2023-03-31.
+ **CTS Code**: ECCD là mã code của chỉ số từ hệ thống CTS (Contemporary Trends and Structures).
+ **CTS Name**: Climate Related Disasters Frequency là tên viết tắt của chỉ số từ hệ thống CTS.
+ **CTS Full Descriptor**: Environment, Climate Change, Climate Indicators, Climate Related Disasters Frequency là Mô tả đầy đủ của chỉ số từ hệ thống CTS.
+ **1990-F2022**: Giá trị

***Kết luận: Sử dụng cột ISO3, Indicator và các cột F1990-F2022 làm dữ liệu về thiên tai của các quốc gia, các cột khác không sử dụng để tính toán.***



## 2.3 Phân tách số liệu của nhiệt độ, nồng độ CO2, diện tích rừng, và tần suất thiên tai theo quốc gia.

### 2.3.1 Trích xuất số liệu nhiệt độ của các quốc gia

In [305]:
l_temp_colname = [str(e) for e in range(1961, 2024, 1)]
np_temp = np.full((len(ISO3), len(l_temp_colname)), 0.0)

for i, iso3 in enumerate(ISO3):
  for j, col in enumerate(l_temp_colname):
    np_temp[i, j] = get_value_by_id_column(d_data['NhietDo'], 'ISO3', iso3, col)

d_exdata['NhietDo'] = pd.DataFrame(np_temp, columns=l_temp_colname, dtype=np.double)
d_exdata['NhietDo'].insert(0, 'ISO3', ISO3)
khai_quat_du_lieu(d_exdata['NhietDo'])


- Kích thước của dữ liệu: (255, 64)

- Tên các cột: ['ISO3', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----

In [306]:
d_exdata['NhietDo']

,ISO3,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,-0.237,0.100,0.020,0.212,-0.430,0.130,-0.268,-0.062,0.531,...,0.818,1.657,0.995,1.157,0.664,0.986,1.285,NaN,NaN,NaN
1,AETMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,-0.126,-0.173,0.844,-0.751,-0.220,0.239,-0.348,-0.398,-0.513,...,0.521,1.204,1.612,1.642,1.624,0.991,0.587,1.475,2.154,1.956
3,AFRTMP,-0.017,-0.036,0.063,-0.156,-0.201,0.138,-0.215,-0.221,0.355,...,1.013,1.190,1.392,1.180,1.178,1.297,1.200,1.396,0.996,1.485
4,AGO,0.060,-0.132,-0.171,-0.210,-0.173,0.198,-0.086,-0.202,0.173,...,0.792,1.326,1.692,0.943,1.306,1.785,1.108,1.401,1.136,1.175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,0.068,-0.011,0.079,0.085,-0.327,0.068,-0.291,-0.209,0.230,...,0.759,0.599,1.129,1.024,0.778,1.129,1.020,0.857,0.845,0.870
251,YEM,0.061,0.020,0.200,-0.215,-0.586,0.122,-0.224,-0.224,0.276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,ZAF,0.246,0.212,0.181,-0.076,-0.030,0.243,-0.080,-0.385,0.274,...,0.814,1.300,1.570,1.140,1.040,1.926,0.960,0.816,1.098,1.121
253,ZMB,0.233,-0.162,-0.385,-0.275,-0.405,0.311,-0.012,-0.125,0.243,...,1.001,2.061,1.575,0.579,0.998,1.349,1.386,1.269,1.062,1.268


### 2.3.2 Trích xuất nồng độ CO2 và diện tích rừng của các quốc gia

#### 2.3.2.1 Phân tích đơn vị trong các cột của tập dữ liệu Forest and Carbon

In [307]:
df_CO2 = d_data['RungCarbon']

print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Forest area")
df_CO2_2 = df_CO2[df_CO2['Indicator']=='Forest area']
print_unique_values_stats(df_CO2_2, 'Unit')
print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Land area")
df_CO2_2 = df_CO2[df_CO2['Indicator']=='Land area']
print_unique_values_stats(df_CO2_2, 'Unit')
print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Share of forest area")
df_CO2_2 = df_CO2[df_CO2['Indicator']=='Share of forest area']
print_unique_values_stats(df_CO2_2, 'Unit')
print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Index of forest extent")
df_CO2_2 = df_CO2[df_CO2['Indicator']=='Index of forest extent']
print_unique_values_stats(df_CO2_2, 'Unit')
print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Carbon stocks in forests")
df_CO2_1 = df_CO2[df_CO2['Indicator']=='Carbon stocks in forests']
print_unique_values_stats(df_CO2_1, 'Unit')
print("-Kiểm tra đơn vị Unit của các phần tử có Indicator là Index of carbon stocks in forests")
df_CO2_2 = df_CO2[df_CO2['Indicator']=='Index of carbon stocks in forests']
print_unique_values_stats(df_CO2_2, 'Unit')


-Kiểm tra đơn vị Unit của các phần tử có Indicator là Forest area
+--------------------+------------+-----------------+
| Giá trị cột Unit   |   Số lượng |   Phần trăm (%) |
+====================+============+=================+
| 1000 HA            |        235 |             100 |
+--------------------+------------+-----------------+
-Kiểm tra đơn vị Unit của các phần tử có Indicator là Land area
+--------------------+------------+-----------------+
| Giá trị cột Unit   |   Số lượng |   Phần trăm (%) |
+====================+============+=================+
| 1000 HA            |        235 |             100 |
+--------------------+------------+-----------------+
-Kiểm tra đơn vị Unit của các phần tử có Indicator là Share of forest area
+--------------------+------------+-----------------+
| Giá trị cột Unit   |   Số lượng |   Phần trăm (%) |
+====================+============+=================+
| Percent            |        235 |             100 |
+--------------------+------------+----

**Kết Luận **

- Trích xuất các dòng dữ liệu có Indicator là Forest area làm dữ liệu diện tích rừng ứng với mã số ISO3 của các quốc gia

- Trích xuất các dòng dữ liệu có Indicator là Carbon stocks in forests, Forest area, Land area để tính toán dữ liệu nồng độ Carbon:

    * Nồng độ Carbon = Carbon stocks in forests / Forest area

    * Nồng độ Carbon = Carbon stocks in forests / Land area

    * Đơn vị sau khi tính là Milion tonnes/HA  (Triệu Tấn / Hecta)





#### 2.3.2.2 * Trích xuất Diện Tích Rừng

In [308]:
l_forest_colname = [str(e) for e in range(1992, 2021, 1)]
np_forest = np.full((len(ISO3), len(l_forest_colname)), 0.0)

DienTichRung = d_data['RungCarbon'][d_data['RungCarbon']['Indicator']=='Forest area']

for i, iso3 in enumerate(ISO3):
  for j, col in enumerate(l_forest_colname):
    np_forest[i, j] = get_value_by_id_column(DienTichRung, 'ISO3', iso3, 'F' + col)

d_exdata['DienTichRung'] = pd.DataFrame(np_forest, columns=l_forest_colname, dtype=np.double)
d_exdata['DienTichRung'].insert(0, 'ISO3', ISO3)
khai_quat_du_lieu(d_exdata['DienTichRung'])

- Kích thước của dữ liệu: (255, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [309]:
d_exdata['DienTichRung']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,0.4200,0.4200,0.4200,0.4200,0.4200,0.4200,0.4200,0.4200,0.4200,...,0.4200,0.4200,0.420,0.4200,0.4200,0.4200,0.42,0.420,0.42,0.42
1,AETMP,945840.6089,950628.7545,950883.6399,951138.5255,951393.4111,951648.2965,951903.1821,952158.0675,953166.9531,...,956338.5036,957498.4138,958658.324,959818.2342,960978.1444,962013.8455,961845.12,961963.710,962100.59,962252.19
2,AFG,1208.4400,1208.4400,1208.4400,1208.4400,1208.4400,1208.4400,1208.4400,1208.4400,1208.4400,...,1208.4400,1208.4400,1208.440,1208.4400,1208.4400,1208.4400,1208.44,1208.440,1208.44,1208.44
3,AFRTMP,685614.0711,702769.4832,699668.6553,696567.8275,693466.9996,690366.1717,687265.3438,684164.5159,681063.6880,...,645042.8380,668200.1500,664293.540,660386.9300,656480.3200,652479.3843,648546.99,644616.828,640606.55,636636.97
4,AGO,78951.9460,78796.5290,78641.1120,78485.6950,78330.2780,78174.8610,78019.4440,77864.0270,77708.6100,...,71602.9380,71047.8760,70492.814,69937.7520,69382.6900,68827.6200,68272.57,67717.510,67162.44,66607.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,175.1660,174.6840,174.2020,173.7200,173.2380,172.7560,172.2740,171.7920,171.3100,...,166.0060,165.5220,165.038,164.5540,164.0700,163.5900,163.11,162.630,162.15,161.67
251,YEM,549.0000,549.0000,549.0000,549.0000,549.0000,549.0000,549.0000,549.0000,549.0000,...,549.0000,549.0000,549.000,549.0000,549.0000,549.0000,549.00,549.000,549.00,549.00
252,ZAF,18069.2900,18032.8900,17996.4900,17960.0900,17923.6900,17887.2900,17850.8900,17814.4900,17778.0900,...,17377.6900,17341.2900,17304.890,17268.4900,17232.0900,17195.6900,17159.29,17122.890,17086.49,17050.09
253,ZMB,47340.4000,47304.6000,47268.8000,47233.0000,47197.2000,47161.4000,47125.6000,47089.8000,47054.0000,...,46507.8200,46319.6400,46131.460,45943.2800,45755.1000,45566.8000,45378.67,45190.460,45002.24,44814.03


#### 2.3.2.3 * Trích xuất Diện Tích Đất

In [310]:
l_land_colname = [str(e) for e in range(1992, 2021, 1)]
np_land = np.full((len(ISO3), len(l_land_colname)), 0.0)

DienTichDat = d_data['RungCarbon'][d_data['RungCarbon']['Indicator']=='Land area']

for i, iso3 in enumerate(ISO3):
  for j, col in enumerate(l_land_colname):
    np_land[i, j] = get_value_by_id_column(DienTichDat, 'ISO3', iso3, 'F' + col)

d_exdata['DienTichDat'] = pd.DataFrame(np_land, columns=l_land_colname, dtype=np.double)
d_exdata['DienTichDat'].insert(0, 'ISO3', ISO3)
khai_quat_du_lieu(d_exdata['DienTichDat'])

- Kích thước của dữ liệu: (255, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

#### 2.3.2.4 * Trích xuất Trữ Lượng Carbon

In [311]:
l_carbon_colname = [str(e) for e in range(1992, 2021, 1)]
np_carbon = np.full((len(ISO3), len(l_carbon_colname)), 0.0)

Carbon = d_data['RungCarbon'][d_data['RungCarbon']['Indicator']=='Carbon stocks in forests']

for i, iso3 in enumerate(ISO3):
  for j, col in enumerate(l_carbon_colname):
    np_carbon[i, j] = get_value_by_id_column(Carbon, 'ISO3', iso3, 'F' + col)

d_exdata['Carbon'] = pd.DataFrame(np_carbon, columns=l_carbon_colname, dtype=np.double)
d_exdata['Carbon'].insert(0, 'ISO3', ISO3)
khai_quat_du_lieu(d_exdata['Carbon'])

- Kích thước của dữ liệu: (255, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [312]:
d_exdata['Carbon']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AETMP,52618.6601,53177.6998,53393.5688,53609.4382,53825.3073,54041.1769,54257.0465,54472.9153,54755.4274,...,56445.5501,56668.6489,56891.7480,57114.8466,57337.9461,57568.1744,57777.8499,57959.7625,58035.0513,58102.5521
2,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFRTMP,55811.7552,57006.7484,56766.9408,56527.1333,56287.3259,56047.5177,55807.7105,55567.9025,55328.0951,...,52532.0036,52890.5790,52578.6891,52266.7998,51954.9100,51666.6077,51265.4387,50960.3994,50654.3608,50343.8299
4,AGO,1364.2896,1361.6040,1358.9184,1356.2328,1353.5472,1350.8616,1348.1760,1345.4904,1342.8048,...,1237.2988,1227.7073,1218.1158,1208.5244,1198.9329,1189.3413,1179.7500,1170.1586,1160.5670,1150.9755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,10.3930,10.3637,10.3344,10.3051,10.2758,10.2465,10.2173,10.1880,10.1587,...,9.8382,9.8085,9.7788,9.7492,9.7195,9.6911,9.6610,9.6326,9.6025,9.5741
251,YEM,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,...,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661,5.1661
252,ZAF,897.4958,895.8663,894.2369,892.6074,890.9779,889.3484,887.7189,886.0895,884.4600,...,866.4846,864.8769,863.2691,861.6614,860.0536,858.4088,856.5918,854.9459,853.2993,851.6520
253,ZMB,1306.5950,1305.6070,1304.6189,1303.6308,1302.6427,1301.6546,1300.6666,1299.6785,1298.6904,...,1283.6158,1278.4221,1273.2283,1268.0345,1262.8408,1257.6437,1252.4513,1247.2567,1242.0618,1236.8672


#### 2.3.2.5 * Trích xuất Nồng Độ Carbon theo Diện Tích Rừng

In [313]:
l_carbon_colname = [str(e) for e in range(1992, 2021, 1)]
np_carbon_forest = np.full((len(ISO3), len(l_carbon_colname)), np.nan)

for i, iso3 in enumerate(ISO3):
  for j, col in enumerate(l_carbon_colname):
    carbon = get_value_by_id_column(d_exdata['Carbon'], 'ISO3', iso3,  col)
    forest = get_value_by_id_column(d_exdata['DienTichRung'], 'ISO3', iso3,  col)
    if carbon is not None and forest is not None and forest != 0.0:
      np_carbon_forest[i, j] = (carbon * 1e6)/forest

d_exdata['Carbon_DienTichRung'] = pd.DataFrame(np_carbon_forest, columns=l_carbon_colname, dtype=np.double)
d_exdata['Carbon_DienTichRung'].insert(0, 'ISO3', ISO3)

khai_quat_du_lieu(d_exdata['Carbon_DienTichRung'])



- Kích thước của dữ liệu: (255, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [314]:
d_exdata['Carbon_DienTichRung']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AETMP,55631.635611,55939.502722,56151.527442,56363.438934,56575.236566,56786.921280,56998.492620,57209.949860,57445.788717,...,59022.563546,59184.065564,59345.177083,59505.898685,59666.233238,59841.315870,60069.806145,60251.506265,60321.188765,60381.834101
2,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFRTMP,81404.039900,81117.279226,81134.034475,81150.939030,81167.994919,81185.202864,81202.567543,81220.088456,81237.769793,...,81439.557972,79153.796957,79149.782339,79145.721130,79141.610825,79185.042383,79046.606476,79055.335180,79072.499025,79077.766879
4,AGO,17279.999660,17279.999732,17279.999805,17279.999878,17279.999951,17280.000025,17280.000098,17280.000173,17280.000247,...,17280.000438,17280.000038,17279.999632,17280.000650,17280.000242,17280.000384,17279.999859,17280.000402,17280.000548,17279.999604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,59332.290513,59328.272767,59324.232787,59320.170389,59316.085385,59311.977587,59308.427273,59304.274937,59300.099235,...,59264.122984,59257.983833,59251.808674,59246.204893,59239.958554,59240.173605,59229.967507,59230.154338,59219.858156,59220.016082
251,YEM,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,...,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215,9410.018215
252,ZAF,49669.677115,49679.574378,49689.517234,49699.494824,49709.512941,49719.571830,49729.671742,49739.818541,49750.001266,...,49861.897640,49873.850215,49885.847295,49897.900743,49909.999309,49919.997395,49920.002518,49930.000134,49939.999380,49950.000264
253,ZMB,27599.999155,27600.000846,27600.000423,27600.000000,27599.999576,27599.999152,27600.000849,27600.000425,27600.000000,...,27599.999312,27600.000777,27600.000087,27599.999391,27600.000874,27600.000439,27600.000176,27600.000089,27599.999467,27599.999375


### 2.3.3 Thu thập số liệu về tần suất thiên tai

#### 2.3.3.1 Cơ sở lý thuyết

Tần suất hàng năm: Đây là cách phổ biến nhất, tính bằng số lần thiên tai xảy ra mỗi năm. Ví dụ, nếu một khu vực trải qua 20 trận bão trong 50 năm, tần suất bão hàng năm sẽ là 20/50 = 0,4 lần/năm.

Đối với tập dữ liệu này thì sẽ tính tần suất các thiên tai từ năm 1980 đến 2022 là 43 năm.

Phân loại thiên tai:

      Climate related disasters frequency, Number of Disasters: TOTAL - Tổng số thảm họa liên quan đến khí hậu.

      Climate related disasters frequency, Number of Disasters: Storm - Số lần xảy ra các cơn bão, bao gồm bão nhiệt đới, lốc xoáy và các hiện tượng thời tiết bão khác.

      Climate related disasters frequency, Number of Disasters: Flood - Số lần xảy ra các trận lụt, bao gồm lụt sông, lụt bờ biển và các hiện tượng lụt khác.

      Climate related disasters frequency, Number of Disasters: Drought - Số lần hạn hán

      Climate related disasters frequency, Number of Disasters: Landslide - Số lần xảy ra lở đất, sạt lở đất do mưa lớn, địa chấn hoặc các nguyên nhân khác.

      Climate related disasters frequency, Number of Disasters: Wildfire - Số lần xảy ra cháy rừng, bao gồm các vụ cháy do thiên nhiên hoặc con người gây ra.

      Climate related disasters frequency, Number of Disasters: Extreme temperature - Số lần xảy ra hiện tượng nhiệt độ cực đoan, bao gồm các đợt nắng nóng, lạnh cực đoan.



#### 2.3.3.2* Trích xuất dữ liệu tần suất tổng số thiên tai của các quốc gia

In [315]:
l_DTotal_colname = [str(e) for e in range(1980, 2023, 1)]
l_FTotal = []
np_dtotal = np.full((len(ISO3), len(l_DTotal_colname)), 0.0)

TongThienTai = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: TOTAL']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DTotal_colname):
    value = get_value_by_id_column(TongThienTai, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dtotal[i, j] = value
    sum += value
  l_FTotal.append(sum/len(l_DTotal_colname))

d_exdata['TongThienTai'] = pd.DataFrame(np_dtotal, columns=l_DTotal_colname, dtype=np.double)
d_exdata['TongThienTai'].insert(0, 'ISO3', ISO3)
d_exdata['TongThienTai'].insert(1, 'TanSuatThienTai', l_FTotal)


khai_quat_du_lieu(d_exdata['TongThienTai'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatThienTai', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+-------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatThienTai   |  1980   |  1981   |  1982   |  1983   |  1984   |  1

In [316]:
d_exdata['TongThienTai']

,ISO3,TanSuatThienTai,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AETMP,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,3.418605,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0,4.0,6.0
3,AFRTMP,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AGO,1.279070,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,0.255814,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
251,YEM,1.116279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,2.0,1.0,2.0,3.0,5.0,2.0,3.0
252,ZAF,2.348837,1.0,2.0,0.0,1.0,2.0,0.0,1.0,1.0,...,1.0,1.0,1.0,3.0,5.0,0.0,6.0,2.0,4.0,8.0
253,ZMB,0.697674,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,1.0


#### 2.3.3.3* Trích xuất dữ liệu tần suất lũ lụt của các quốc gia

In [317]:
l_DFlood_colname = [str(e) for e in range(1980, 2023, 1)]
l_FFlood = []
np_dflood = np.full((len(ISO3), len(l_DFlood_colname)), 0.0)

LuLut = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Flood']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DFlood_colname):
    value = get_value_by_id_column(LuLut, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dflood[i, j] = value
    sum += value
  l_FFlood.append(sum/len(l_DFlood_colname))

d_exdata['LuLut'] = pd.DataFrame(np_dflood, columns=l_DFlood_colname, dtype=np.double)
d_exdata['LuLut'].insert(0, 'ISO3', ISO3)
d_exdata['LuLut'].insert(1, 'TanSuatLuLut', l_FFlood)


khai_quat_du_lieu(d_exdata['LuLut'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatLuLut', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatLuLut   |  1980   |  1981   |  1982   |  1983   |  1984   |  1985   |  

In [318]:
d_exdata['LuLut']

,ISO3,TanSuatLuLut,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ABW,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AETMP,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,2.255814,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,AFRTMP,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AGO,1.046512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,4.0,1.0,1.0,3.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,WSM,0.023256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,YEM,0.906977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,3.0,5.0,2.0,3.0
252,ZAF,1.046512,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,2.0,1.0,0.0,4.0,1.0,1.0,8.0
253,ZMB,0.511628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0


#### 2.3.3.4* Trích xuất dữ liệu tần suất hạn hán của các quốc gia

In [319]:
l_DDrought_colname = [str(e) for e in range(1980, 2023, 1)]
l_DDrought = []
np_ddrought = np.full((len(ISO3), len(l_DDrought_colname)), 0.0)

HanHan = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Drought']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DDrought_colname):
    value = get_value_by_id_column(HanHan, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_ddrought[i, j] = value
    sum += value
  l_DDrought.append(sum/len(l_DDrought_colname))

d_exdata['HanHan'] = pd.DataFrame(np_ddrought, columns=l_DDrought_colname, dtype=np.double)
d_exdata['HanHan'].insert(0, 'ISO3', ISO3)
d_exdata['HanHan'].insert(1, 'TanSuatHanHan', l_DDrought)


khai_quat_du_lieu(d_exdata['HanHan'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatHanHan', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+-----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatHanHan   |  1980   |  1981   |  1982   |  1983   |  1984   |  1985   

#### 2.3.3.5* Trích xuất dữ liệu tần suất cháy rừng của các quốc gia

In [320]:
l_DWildfire_colname = [str(e) for e in range(1980, 2023, 1)]
l_DWildfire = []
np_dWildfire = np.full((len(ISO3), len(l_DWildfire_colname)), 0.0)

ChayRung = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Wildfire']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DWildfire_colname):
    value = get_value_by_id_column(ChayRung, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dWildfire[i, j] = value
    sum += value
  l_DWildfire.append(sum/len(l_DWildfire_colname))

d_exdata['ChayRung'] = pd.DataFrame(np_dWildfire, columns=l_DWildfire_colname, dtype=np.double)
d_exdata['ChayRung'].insert(0, 'ISO3', ISO3)
d_exdata['ChayRung'].insert(1, 'TanSuatChayRung', l_DWildfire)


khai_quat_du_lieu(d_exdata['ChayRung'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatChayRung', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+-------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatChayRung   |  1980   |  1981   |  1982   |  1983   |  1984   |  1

#### 2.3.3.6* Trích xuất dữ liệu tần suất bão của các quốc gia

In [321]:
l_DStorm_colname = [str(e) for e in range(1980, 2023, 1)]
l_DStorm = []
np_dStorm = np.full((len(ISO3), len(l_DStorm_colname)), 0.0)

Bao = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Storm']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DStorm_colname):
    value = get_value_by_id_column(Bao, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dStorm[i, j] = value
    sum += value
  l_DStorm.append(sum/len(l_DStorm_colname))

d_exdata['Bao'] = pd.DataFrame(np_dStorm, columns=l_DStorm_colname, dtype=np.double)
d_exdata['Bao'].insert(0, 'ISO3', ISO3)
d_exdata['Bao'].insert(1, 'TanSuatBao', l_DStorm)


khai_quat_du_lieu(d_exdata['Bao'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatBao', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+--------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatBao   |  1980   |  1981   |  1982   |  1983   |  1984   |  1985   |  1986  

#### 2.3.3.7* Trích xuất dữ liệu tần suất sạt lở của các quốc gia

In [322]:
l_DLandslide_colname = [str(e) for e in range(1980, 2023, 1)]
l_DLandslide = []
np_dLandslide = np.full((len(ISO3), len(l_DLandslide_colname)), 0.0)

SatLo = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Landslide']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DLandslide_colname):
    value = get_value_by_id_column(SatLo, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dLandslide[i, j] = value
    sum += value
  l_DLandslide.append(sum/len(l_DLandslide_colname))

d_exdata['SatLo'] = pd.DataFrame(np_dLandslide, columns=l_DLandslide_colname, dtype=np.double)
d_exdata['SatLo'].insert(0, 'ISO3', ISO3)
d_exdata['SatLo'].insert(1, 'TanSuatSatLo', l_DLandslide)


khai_quat_du_lieu(d_exdata['SatLo'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatSatLo', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatSatLo   |  1980   |  1981   |  1982   |  1983   |  1984   |  1985   |  

#### 2.3.3.8* Trích xuất dữ liệu tần suất nhiệt độ cực đoan của các quốc gia

In [323]:
l_DExtreme_temperature_colname = [str(e) for e in range(1980, 2023, 1)]
l_DExtreme_temperature = []
np_dExtreme_temperature = np.full((len(ISO3), len(l_DExtreme_temperature_colname)), 0.0)

NhietDoCucDoan = d_data['ThienTai'][d_data['ThienTai']['Indicator']=='Climate related disasters frequency, Number of Disasters: Extreme temperature']


for i, iso3 in enumerate(ISO3):
  sum = 0.0
  for j, col in enumerate(l_DExtreme_temperature_colname):
    value = get_value_by_id_column(NhietDoCucDoan, 'ISO3', iso3, col)
    value = 0 if value is None else value
    np_dExtreme_temperature[i, j] = value
    sum += value
  l_DExtreme_temperature.append(sum/len(l_DExtreme_temperature_colname))

d_exdata['NhietDoCucDoan'] = pd.DataFrame(np_dExtreme_temperature, columns=l_DExtreme_temperature_colname, dtype=np.double)
d_exdata['NhietDoCucDoan'].insert(0, 'ISO3', ISO3)
d_exdata['NhietDoCucDoan'].insert(1, 'TanSuatNhietDoCucDoan', l_DExtreme_temperature)


khai_quat_du_lieu(d_exdata['NhietDoCucDoan'])

- Kích thước của dữ liệu: (255, 45)

- Tên các cột: ['ISO3', 'TanSuatNhietDoCucDoan', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

- Kiểu dữ liệu của các cột: 
+---------+-------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   | TanSuatNhietDoCucDoan   |  1980   |  1981   |  1982   |  198

### 2.3.4* Tổng Kết

.........................

In [324]:
key_data_world = ['NhietDo', 'DienTichRung', 'DienTichDat', 'Carbon',
                 'Carbon_DienTichRung', 'Carbon_DienTichDat',
                 'TongThienTai', 'LuLut', 'HanHan']

## 2.4 Phân tách dữ liệu mực nước biển

In [325]:
key_vung_bien = sorted(d_data['MucNuoc']['Measure'].unique().tolist())
thoigian_mucnuoc = [str(e) for e in range(1992, 2025, 1)]

try:
  d_data['MucNuoc']['Date'] = d_data['MucNuoc']['Date'].str.replace('D', '')
  d_data['MucNuoc']['Date'] = pd.to_datetime(d_data['MucNuoc']['Date'], format='%m/%d/%Y')
except AttributeError:
  pass

np_mucnuoc = np.full((len(key_vung_bien), len(thoigian_mucnuoc)), np.nan)

for i, iso3 in enumerate(key_vung_bien):
  for j, col in enumerate(thoigian_mucnuoc):
    temp_df = d_data['MucNuoc'][(d_data['MucNuoc']['Measure'] == iso3) & (d_data['MucNuoc']['Date'].dt.year == int(col))]
    np_mucnuoc[i, j] = temp_df['Value'].mean(skipna=True)

d_exdata['MucNuoc'] = pd.DataFrame(np_mucnuoc, columns=thoigian_mucnuoc, dtype=np.double)
d_exdata['MucNuoc'].insert(0, 'ISO3', key_vung_bien)

khai_quat_du_lieu(d_exdata['MucNuoc'])

- Kích thước của dữ liệu: (25, 34)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |  2

In [326]:
d_exdata['MucNuoc']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Adriatic Sea,-4.32,-43.352353,-14.461176,-18.405484,3.912353,7.342121,11.233030,7.281515,20.412353,...,41.870541,40.263000,5.071800,67.494595,48.928571,49.055676,43.623243,40.335890,67.538571,77.761429
1,Andaman Sea,-12.13,-11.821176,-45.074118,1.073125,13.407879,-76.146667,-15.164848,13.432121,30.172941,...,46.638378,118.080429,48.602549,59.548378,9.718649,86.420556,128.116486,120.405676,32.191429,20.615714
2,Arabian Sea,3.09,-1.883529,-1.110000,3.727500,-6.663030,11.391515,3.440000,-28.796364,-19.848235,...,76.339189,76.094143,66.067843,46.890811,81.558378,94.704865,92.790000,74.707568,88.901429,79.972857
3,Atlantic Ocean,-12.31,-18.151176,-14.660000,-7.892258,-6.821765,-4.066061,0.809697,-2.917576,-1.501176,...,45.719189,49.254000,54.032353,60.154865,68.727838,67.125946,73.136081,73.060405,80.109286,75.495714
4,Baltic Sea,216.55,-36.814706,-12.332353,25.759375,-108.362121,-1.774242,34.204545,-7.334848,31.441176,...,143.338378,56.301714,119.694902,34.972973,80.689189,176.454865,71.869054,126.682329,122.545000,191.640000
5,Bay Bengal,0.78,-9.378824,-20.293529,1.808125,4.269394,-26.209394,-9.927576,-4.909394,12.485882,...,66.620000,99.705286,62.379412,48.374865,46.020811,87.056757,128.065541,105.695068,57.572143,17.611429
6,Bering Sea,-25.09,-8.007647,-9.834118,16.145484,35.701176,52.658485,12.934242,-25.541515,-30.348824,...,63.991351,65.656286,49.166667,57.709189,48.995676,52.171081,37.193378,55.908243,60.643571,40.464286
7,Caribbean Sea,-3.73,-11.368235,6.240588,-2.842903,-17.177059,-3.763333,-1.775455,-7.508788,-19.612353,...,73.442432,73.216571,39.422549,57.316486,84.367838,80.734054,70.994189,93.469041,101.337857,59.621429
8,Gulf Mexico,-7.37,-7.881765,-15.622941,-0.979375,-31.783636,-14.213939,-5.138182,15.337576,8.515294,...,80.267838,98.061571,74.394510,79.359459,108.878378,110.229189,99.437297,128.625479,133.255714,34.782857
9,Indian Ocean,-19.32,-19.002353,-16.178824,-10.155484,-9.734706,-5.538182,2.931515,-5.492727,-3.858235,...,56.091351,47.808000,51.532941,52.160000,62.378919,65.412703,63.429595,73.045811,77.902143,87.905714


#3 Phân tách dữ liệu các nước Đông Nam Á, Đông Á

## 3.0* Cơ Sở Lý Thuyết


**- Danh sách các nước Đông Nam Á cùng với mã ISO3 của từng quốc gia:**

Brunei Darussalam - BRN, Cambodia (Campuchia) - KHM, Indonesia - IDN, Laos (Lào) - LAO, Malaysia - MYS, Myanmar (Miến Điện) - MMR, Philippines (Philippin) - PHL, Singapore (Singapore) - SGP, Thailand (Thái Lan) - THA, Timor-Leste (Đông Timor) - TLS, Vietnam (Việt Nam) - VNM

**- Danh sách các nước Đông Á cùng với mã ISO3 của từng quốc gia:**

China (Trung Quốc) - CHN, Japan (Nhật Bản) - JPN, Mongolia (Mông Cổ) - MNG
North Korea (Triều Tiên) - PRK, South Korea (Hàn Quốc) - KOR, Taiwan (Đài Loan) - TWN

**- Các mốc thời gian ở các tập dữ liệu:**

    Nhiệt Độ: 1961 -> 2023

    Diện Tích Rừng: 1992 -> 2020

    Nồng Độ CO2: 1992 -> 2020

    Tổng Thiên Tai: 1980 -> 2022

In [327]:
d_data_dna, d_data_donga = {}, {}
l_timeline = [str(e) for e in range(1992, 2021, 1)]

ma_iso_dna = sorted(["BRN", "KHM", "IDN", "LAO", "MYS", "MMR", "PHL", "SGP", "THA", "TLS", "VNM"])
ma_iso_donga = sorted(["CHN", "JPN", "MNG", "PRK", "KOR"])
# ma_iso_donga khong chua TWN vì TWN thiếu dữ liệu về diện tích rừng nên không thể tính toán được

vung_bien_tiep_giap_viet_nam = sorted(['South China', 'Gulf Mexico'])
vung_bien_tiep_giap_dong_nam_a = sorted(['Andaman Sea', 'Bay Bengal', 'Indonesian', 'South China'])
vung_bien_tiep_giap_dong_a = sorted(['Sea Japan', 'South China', 'Yellow Sea'])

## 3.1* Tiền xử lý dữ liệu Nhiệt Độ của khu vực Đông Nam Á và Đông Á

### 3.1.1* Đông Nam Á

In [328]:
# Trích xuất tập dữ liệu

np_nhietdo_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_nhietdo_dna[i, j] = get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', iso3, col)

d_data_dna['NhietDo'] = pd.DataFrame(np_nhietdo_dna, columns=l_timeline, dtype=np.double)
d_data_dna['NhietDo'].insert(0, 'ISO3', ma_iso_dna)

in_vi_tri_gia_tri_thieu(d_data_dna['NhietDo'], 'ISO3')

+----+------+-----------+
|    | ID   |   Tên Cột |
+====+======+===========+
|  0 | SGP  |      1992 |
+----+------+-----------+
|  1 | TLS  |      1992 |
+----+------+-----------+
|  2 | TLS  |      1993 |
+----+------+-----------+
|  3 | TLS  |      1994 |
+----+------+-----------+
|  4 | SGP  |      2000 |
+----+------+-----------+
|  5 | TLS  |      2000 |
+----+------+-----------+
|  6 | TLS  |      2001 |
+----+------+-----------+
|  7 | SGP  |      2002 |
+----+------+-----------+
|  8 | TLS  |      2002 |
+----+------+-----------+
|  9 | SGP  |      2003 |
+----+------+-----------+
| 10 | SGP  |      2007 |
+----+------+-----------+
| 11 | SGP  |      2008 |
+----+------+-----------+


In [329]:
d_data_dna['NhietDo']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,BRN,0.273,0.150,0.341,0.315,0.233,0.486,1.124,0.350,0.286,...,0.374,0.596,0.779,0.712,0.932,1.383,0.942,0.903,1.221,1.222
1,IDN,0.225,0.246,0.347,0.372,0.400,0.405,0.992,0.260,0.265,...,0.288,0.420,0.773,0.759,0.755,1.386,0.932,0.846,1.026,1.192
2,KHM,0.444,0.317,0.409,0.626,0.289,0.335,1.275,0.343,0.186,...,0.354,0.857,0.999,0.549,1.039,1.331,0.994,0.655,1.370,1.275
3,LAO,0.208,0.301,0.416,0.635,0.148,0.335,1.421,0.649,0.022,...,0.103,0.942,1.121,0.688,1.284,1.306,1.087,0.788,2.014,1.546
4,MMR,0.028,0.097,0.473,0.614,0.405,0.224,1.163,0.864,0.193,...,0.489,0.889,0.947,0.835,1.160,1.440,1.343,1.087,1.651,1.547
5,MYS,0.316,0.164,0.310,0.411,0.341,0.538,1.122,0.335,0.306,...,0.517,0.697,0.876,0.891,1.046,1.484,1.022,1.026,1.318,1.281
6,PHL,0.407,0.404,0.481,0.528,0.432,0.505,1.331,0.558,0.669,...,0.621,0.881,1.024,0.905,1.105,1.563,1.208,1.242,1.318,1.470
7,SGP,NaN,0.193,0.558,0.439,0.201,0.655,0.872,0.247,NaN,...,0.685,1.223,1.000,1.328,1.596,1.782,1.158,0.992,1.626,1.407
8,THA,0.358,0.294,0.353,0.681,0.228,0.388,1.424,0.409,0.042,...,0.247,0.908,1.109,0.692,1.202,1.462,1.044,0.813,1.714,1.475
9,TLS,NaN,NaN,NaN,0.165,0.206,-0.063,0.825,-0.038,NaN,...,-0.236,-0.273,0.527,0.465,0.525,1.413,0.742,0.333,0.443,1.351


In [330]:
# Tính ma trận tương quan

d_nhietdo_tuongquan = d_exdata['NhietDo'].drop(columns=['ISO3'])
d_nhietdo_tuongquan = d_nhietdo_tuongquan.apply(lambda row: row.fillna(row.mean()), axis=1)
np_nhietdo_tuongquan = d_nhietdo_tuongquan.to_numpy().T
d_nhietdo_tuongquan = pd.DataFrame(np_nhietdo_tuongquan, columns=ISO3, dtype=np.double)
d_nhietdo_tuongquan = d_nhietdo_tuongquan.dropna(axis=1)
correlation_nhietdo = d_nhietdo_tuongquan.corr()
corr_tls = correlation_nhietdo['TLS'][(correlation_nhietdo['TLS'] < -0.61) | (correlation_nhietdo['TLS'] > 0.61)]
corr_sgp = correlation_nhietdo['SGP'][(correlation_nhietdo['SGP'] < -0.54) | (correlation_nhietdo['SGP'] > 0.54)]
col_tls = corr_tls.to_frame().transpose().columns.to_list()
col_tls.remove('TLS')
col_sgp = corr_sgp.to_frame().transpose().columns.to_list()
col_sgp.remove('SGP')
print("- Các nước có độ tương quan cao với TLS so với các nước khác")
print(col_tls)
in_tuong_quan(corr_tls)
print("\n- Các nước có độ tương quan cao với SGP so với các nước khác")
print(col_sgp)
in_tuong_quan(corr_sgp)

- Các nước có độ tương quan cao với TLS so với các nước khác
['AUS', 'COD', 'GUF', 'HND', 'IDN', 'MYS', 'OCETMP', 'PHL', 'PNG', 'THA']
+-----+----------+----------+----------+----------+---------+----------+----------+----------+----------+----------+-------+
|     |      AUS |      COD |      GUF |      HND |     IDN |      MYS |   OCETMP |      PHL |      PNG |      THA |   TLS |
|-----+----------+----------+----------+----------+---------+----------+----------+----------+----------+----------+-------|
| TLS | 0.636618 | 0.631629 | 0.612606 | 0.611622 | 0.68222 | 0.625342 | 0.647883 | 0.623546 | 0.629295 | 0.624546 |     1 |
+-----+----------+----------+----------+----------+---------+----------+----------+----------+----------+----------+-------+

- Các nước có độ tương quan cao với SGP so với các nước khác
['BGR', 'BIH', 'BRN', 'CZE', 'EURTMP', 'FSM', 'HRV', 'HUN', 'MHL', 'MKD', 'MUS', 'MYS', 'PLW', 'POL', 'ROU', 'SVK', 'SVN']
+-----+----------+----------+----------+---------+-----

In [331]:
# Thêm giá trị thiếu
def tinh_trung_binh_cua_hang(df, cot_id, id):
  value = (df[df[cot_id] == id].drop(columns=cot_id)).mean(axis=1, skipna=True)
  return value.iloc[0]

mean_row_tls_temp = tinh_trung_binh_cua_hang(d_exdata['NhietDo'], 'ISO3', 'TLS')

mean_col_tls_temp_1992 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '1992') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '1992'] = (mean_row_tls_temp + mean_col_tls_temp_1992)/2

mean_col_tls_temp_1993 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '1993') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '1993'] = (mean_row_tls_temp + mean_col_tls_temp_1993)/2

mean_col_tls_temp_1994 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '1994') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '1994'] = (mean_row_tls_temp + mean_col_tls_temp_1994)/2

mean_col_tls_temp_2000 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2000') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '2000'] = (mean_row_tls_temp + mean_col_tls_temp_2000)/2

mean_col_tls_temp_2001 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2001') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '2001'] = (mean_row_tls_temp + mean_col_tls_temp_2001)/2

mean_col_tls_temp_2002 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2002') for e in col_tls])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'TLS', '2002'] = (mean_row_tls_temp + mean_col_tls_temp_2002)/2


mean_row_sgp_temp = tinh_trung_binh_cua_hang(d_exdata['NhietDo'], 'ISO3', 'SGP')

#mean_col_sgp_temp_1992 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '1992') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '1992'] = mean_row_sgp_temp

mean_col_sgp_temp_2000 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2000') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '2000'] = (mean_row_sgp_temp + mean_col_sgp_temp_2000)/2

mean_col_sgp_temp_2002 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2002') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '2002'] = (mean_row_sgp_temp + mean_col_sgp_temp_2002)/2

mean_col_sgp_temp_2003 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2003') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '2003'] = (mean_row_sgp_temp + mean_col_sgp_temp_2003)/2

mean_col_sgp_temp_2007 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2007') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '2007'] = (mean_row_sgp_temp + mean_col_sgp_temp_2007)/2

#mean_col_sgp_temp_2008 = np.nanmean([get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', e, '2008') for e in col_sgp])
d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'SGP', '2008'] = mean_row_sgp_temp

khai_quat_du_lieu(d_data_dna['NhietDo'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [332]:
np_trungbinh_nhietdo_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['NhietDo'], 'ISO3', iso3, col)
  np_trungbinh_nhietdo_dna[0, j] = sum/len(ma_iso_dna)
d_data_dna['TrungBinhNhietDo'] = pd.DataFrame(np_trungbinh_nhietdo_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TrungBinhNhietDo'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TrungBinhNhietDo'], True)
d_data_dna['TrungBinhNhietDo']


+----------+----------+----------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+=========+=========+=========+============+============+============+
| 0.700632 | 0.353607 | 0.250484 | 0.397494 |       0.615364 | 0.916636 | 1.42627 | 20.3183 |      29 |   0.125038 |   0.685399 |  -0.474927 |
+----------+----------+----------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.306905,0.250484,0.397494,0.468636,0.263364,0.375091,1.168727,0.428545,0.29999,0.554748,...,0.324545,0.718545,0.916636,0.761364,1.074909,1.426273,1.052273,0.860455,1.403182,1.380364


**Nhận Xét**

In [333]:
# Chuẩn Hóa Min Max , min là 0.0, max là 1.6
d_data_dna['TrungBinhNhietDo_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TrungBinhNhietDo'].iloc[0], 0.0, 1.6), columns=d_data_dna['TrungBinhNhietDo'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TrungBinhNhietDo_ChuanHoa'], row_index=0)
d_data_dna['TrungBinhNhietDo_ChuanHoa']


+----------+----------+----------+----------+----------------+----------+---------+--------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |     max |    sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+=========+========+=========+============+============+============+
| 0.437895 | 0.221004 | 0.156553 | 0.248433 |       0.384602 | 0.572898 | 0.89142 | 12.699 |      29 |  0.0488428 |   0.685399 |  -0.474927 |
+----------+----------+----------+----------+----------------+----------+---------+--------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.191816,0.156553,0.248433,0.292898,0.164602,0.234432,0.730455,0.267841,0.187494,0.346718,...,0.202841,0.449091,0.572898,0.475852,0.671818,0.89142,0.65767,0.537784,0.876989,0.862727


### 3.1.2* Đông Á

In [334]:
# Trích xuất tập dữ liệu

np_nhietdo_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_nhietdo_donga[i, j] = get_value_by_id_column(d_exdata['NhietDo'], 'ISO3', iso3, col)

d_data_donga['NhietDo'] = pd.DataFrame(np_nhietdo_donga, columns=l_timeline, dtype=np.double)
d_data_donga['NhietDo'].insert(0, 'ISO3', ma_iso_donga)

in_vi_tri_gia_tri_thieu(d_data_donga['NhietDo'], 'ISO3')

In [335]:
d_data_donga['NhietDo']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,CHN,0.146,0.299,0.752,0.598,0.232,0.758,1.290,1.256,0.599,...,0.808,0.639,1.088,1.062,1.302,1.333,1.588,1.375,1.411,1.580
1,JPN,0.326,0.005,0.958,0.342,-0.132,0.449,1.194,0.944,0.624,...,0.585,0.381,0.609,0.543,0.817,1.307,0.781,0.908,1.253,1.418
2,KOR,0.614,0.035,1.248,0.413,-0.026,0.728,1.482,1.063,0.446,...,0.347,0.430,0.538,1.024,0.973,1.652,1.462,0.964,1.521,1.498
3,MNG,0.861,0.508,1.347,1.284,0.430,1.966,2.151,1.797,1.013,...,0.804,0.277,1.060,1.910,2.161,1.425,2.534,2.236,1.871,2.384
4,PRK,0.753,0.554,1.341,0.681,0.259,0.968,1.750,1.354,0.569,...,0.503,0.407,0.288,1.575,1.280,1.495,1.671,0.935,1.889,1.892


Nhận xét: Nhiệt độ ở các nước Đông Á không bị thiếu qua từng năm.

In [336]:
np_trungbinh_nhietdo_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['NhietDo'], 'ISO3', iso3, col)
  np_trungbinh_nhietdo_donga[0, j] = sum/len(ma_iso_donga)
d_data_donga['TrungBinhNhietDo'] = pd.DataFrame(np_trungbinh_nhietdo_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TrungBinhNhietDo'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TrungBinhNhietDo'], True)
d_data_donga['TrungBinhNhietDo']

+---------+----------+--------+--------+----------------+--------+--------+---------+---------+------------+------------+------------+
|    mean |      std |    min |    25% |   50% (median) |    75% |    max |     sum |   count |   variance |   skewness |   kurtosis |
+=========+==========+========+========+================+========+========+=========+=========+============+============+============+
| 1.01937 | 0.434033 | 0.1526 | 0.6502 |         1.0132 | 1.3066 | 1.7544 | 29.5616 |      29 |   0.188384 |  -0.136278 |  -0.879091 |
+---------+----------+--------+--------+----------------+--------+--------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.54,0.2802,1.1292,0.6636,0.1526,0.9738,1.5734,1.2828,0.6502,0.9852,...,0.6094,0.4268,0.7166,1.2228,1.3066,1.4424,1.6072,1.2836,1.589,1.7544


In [337]:
# Chuẩn Hóa Min Max , min là 0.09, max là 1.8
d_data_donga['TrungBinhNhietDo_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TrungBinhNhietDo'].iloc[0], 0.09, 1.8), columns=d_data_donga['TrungBinhNhietDo'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TrungBinhNhietDo_ChuanHoa'], row_index=0)
d_data_donga['TrungBinhNhietDo_ChuanHoa']

+----------+---------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |     std |       min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+=========+===========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.543489 | 0.25382 | 0.0366082 | 0.327602 |       0.539883 | 0.711462 | 0.973333 | 15.7612 |      29 |  0.0644247 |  -0.136278 |  -0.879091 |
+----------+---------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.263158,0.111228,0.607719,0.335439,0.036608,0.516842,0.867485,0.697544,0.327602,0.523509,...,0.303743,0.196959,0.366433,0.662456,0.711462,0.790877,0.887251,0.698012,0.876608,0.973333


## 3.2* Tiền xử lý dữ liệu Diện Tích Rừng của khu vực Đông Nam Á và Đông Á

### 3.2.1 Đông Nam Á

In [338]:
# Trích xuất tập dữ liệu

np_dtrung_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_dtrung_dna[i, j] = get_value_by_id_column(d_exdata['DienTichRung'], 'ISO3', iso3, col)

d_data_dna['DienTichRung'] = pd.DataFrame(np_dtrung_dna, columns=l_timeline, dtype=np.double)
d_data_dna['DienTichRung'].insert(0, 'ISO3', ma_iso_dna)

in_vi_tri_gia_tri_thieu(d_data_dna['DienTichRung'], 'ISO3')
khai_quat_du_lieu(d_data_dna['DienTichRung'])


- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [339]:
d_data_dna['DienTichRung']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,BRN,409.8000,408.2000,406.6000,405.0000,403.4000,401.8000,400.2000,398.6000,397.0000,...,380.0000,380.0000,380.000,380.0000,380.0000,380.0000,380.00,380.00,380.00,380.00
1,IDN,115092.0000,113365.5000,111639.0000,109912.5000,108186.0000,106459.5000,104733.0000,103006.5000,101280.0000,...,98732.9400,97806.6800,96880.420,95954.1600,95027.9000,95271.8000,93949.80,93344.27,92738.73,92133.20
2,KHM,10960.0320,10937.6530,10915.2740,10892.8950,10870.5160,10848.1370,10825.7580,10803.3790,10781.0000,...,10240.7480,9892.2660,9543.784,9195.3020,8846.8200,8691.1300,8535.44,8379.75,8224.06,8068.37
3,LAO,17759.4000,17717.6000,17675.8000,17634.0000,17592.2000,17550.4000,17508.6000,17466.8000,17425.0000,...,16906.0000,16871.5000,16837.000,16802.5000,16768.0000,16733.5000,16699.00,16664.50,16630.00,16595.50
4,MMR,38348.4060,37913.3690,37478.3320,37043.2950,36608.2580,36173.2210,35738.1840,35303.1470,34868.1100,...,31151.2880,30861.5760,30571.864,30282.1520,29992.4400,29702.7300,29413.02,29123.31,28833.60,28543.89
5,MYS,20433.0700,20340.3550,20247.6400,20154.9250,20062.2100,19969.4950,19876.7800,19784.0650,19691.3500,...,19050.9640,19154.2780,19257.592,19360.9060,19464.2200,19314.6400,19264.49,19214.34,19164.19,19114.04
6,PHL,7684.9000,7637.9450,7590.9900,7544.0350,7497.0800,7450.1250,7403.1700,7356.2150,7309.2600,...,6874.6060,6909.4920,6944.378,6979.2640,7014.1500,7049.0400,7083.93,7118.81,7153.70,7188.59
7,SGP,15.2642,15.4829,15.7015,15.9202,16.1389,16.3575,16.5762,16.7948,17.0135,...,17.4866,17.2328,16.979,16.7252,16.4714,16.2855,16.11,15.93,15.75,15.57
8,THA,19288.4000,19252.1000,19215.8000,19179.5000,19143.2000,19106.9000,19070.6000,19034.3000,18998.0000,...,20070.6000,20068.2000,20065.800,20063.4000,20061.0000,20017.0000,19981.00,19945.00,19909.00,19873.00
9,TLS,960.3000,958.9000,957.5000,956.1000,954.7000,953.3000,951.9000,950.5000,949.1000,...,933.7000,932.3000,930.900,929.5000,928.1000,926.7000,925.60,923.90,922.50,921.10


In [340]:
np_tong_dtrung_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['DienTichRung'], 'ISO3', iso3, col)
  np_tong_dtrung_dna[0, j] = sum
d_data_dna['TongDienTichRung'] = pd.DataFrame(np_tong_dtrung_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongDienTichRung'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongDienTichRung'], True)
d_data_dna['TongDienTichRung']

+--------+---------+--------+--------+----------------+--------+--------+-------------+---------+-------------+------------+------------+
|   mean |     std |    min |    25% |   50% (median) |    75% |    max |         sum |   count |    variance |   skewness |   kurtosis |
+========+=========+========+========+================+========+========+=============+=========+=============+============+============+
| 221575 | 8941.66 | 207476 | 215221 |         220927 | 225664 | 240809 | 6.42567e+06 |      29 | 7.99532e+07 |   0.490821 |  -0.307597 |
+--------+---------+--------+--------+----------------+--------+--------+-------------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,240809.1602,238645.5069,236481.8535,234318.2002,232154.5469,229990.8935,227827.2402,225663.5868,223499.9335,223071.0601,...,217881.1526,216551.1048,215221.057,213891.0092,212560.9614,212480.5055,210663.78,209601.1,208538.72,207476.35


In [341]:
# Chuẩn Hóa Min Max , min là 200000, max là 250000
d_data_dna['TongDienTichRung_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongDienTichRung'].iloc[0], 200000, 250000), columns=d_data_dna['TongDienTichRung'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongDienTichRung_ChuanHoa'], row_index=0)
d_data_dna['TongDienTichRung_ChuanHoa']


+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.431495 | 0.178833 | 0.149527 | 0.304421 |       0.418534 | 0.513272 | 0.816183 | 12.5134 |      29 |  0.0319813 |   0.490821 |  -0.307597 |
+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.816183,0.77291,0.729637,0.686364,0.643091,0.599818,0.556545,0.513272,0.469999,0.461421,...,0.357623,0.331022,0.304421,0.27782,0.251219,0.24961,0.213276,0.192022,0.170774,0.149527


### 3.2.2 Đông Á

In [342]:
# Trích xuất tập dữ liệu

np_dtrung_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_dtrung_donga[i, j] = get_value_by_id_column(d_exdata['DienTichRung'], 'ISO3', iso3, col)

d_data_donga['DienTichRung'] = pd.DataFrame(np_dtrung_donga, columns=l_timeline, dtype=np.double)
d_data_donga['DienTichRung'].insert(0, 'ISO3', ma_iso_donga)

in_vi_tri_gia_tri_thieu(d_data_donga['DienTichRung'], 'ISO3')
khai_quat_du_lieu(d_data_donga['DienTichRung'])


- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [343]:
d_data_donga['DienTichRung']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,CHN,161112.582,163098.578,165084.574,167070.57,169056.566,171042.562,173028.558,175014.554,177000.55,...,202547.154,204483.928,206420.702,208357.476,210294.25,212459.867,214339.47,216219.04,218098.61,219978.18
1,JPN,24935.200,24927.800,24920.400,24913.00,24905.600,24898.200,24890.800,24883.400,24876.00,...,24961.600,24957.200,24952.800,24948.400,24944.00,24940.000,24935.00,24935.00,24935.00,24935.00
2,KOR,6536.000,6528.500,6521.000,6513.50,6506.000,6498.500,6491.000,6483.500,6476.00,...,6377.000,6367.000,6357.000,6347.000,6337.00,6325.000,6317.00,6307.00,6297.00,6287.00
3,MNG,14334.380,14325.570,14316.760,14307.95,14299.140,14290.330,14281.520,14272.710,14263.90,...,14182.786,14181.672,14180.558,14179.444,14178.33,14177.220,14176.11,14175.00,14173.89,14172.78
4,PRK,6820.562,6774.828,6729.094,6683.36,6637.626,6591.892,6546.158,6500.424,6454.69,...,6221.160,6199.930,6178.700,6157.470,6136.24,6115.010,6093.78,6072.55,6051.32,6030.09


In [344]:
np_tong_dtrung_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['DienTichRung'], 'ISO3', iso3, col)
  np_tong_dtrung_donga[0, j] = sum
d_data_donga['TongDienTichRung'] = pd.DataFrame(np_tong_dtrung_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongDienTichRung'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongDienTichRung'], True)
d_data_donga['TongDienTichRung']

+--------+---------+--------+--------+----------------+--------+--------+------------+---------+-------------+------------+------------+
|   mean |     std |    min |    25% |   50% (median) |    75% |    max |        sum |   count |    variance |   skewness |   kurtosis |
+========+=========+========+========+================+========+========+============+=========+=============+============+============+
| 242779 | 18065.7 | 213739 | 227155 |         243062 | 258090 | 271403 | 7.0406e+06 |      29 | 3.26368e+08 | -0.0233803 |   -1.30812 |
+--------+---------+--------+--------+----------------+--------+--------+------------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,213738.724,215655.276,217571.828,219488.38,221404.932,223321.484,225238.036,227154.588,229071.14,231402.993,...,254289.7,256189.73,258089.76,259989.79,261889.82,264017.097,265861.36,267708.59,269555.82,271403.05


In [345]:
# Chuẩn Hóa Min Max , min là 212400, max là 278000
d_data_donga['TongDienTichRung_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongDienTichRung'].iloc[0], 212400, 278000), columns=d_data_donga['TongDienTichRung'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongDienTichRung_ChuanHoa'], row_index=0)
d_data_donga['TongDienTichRung_ChuanHoa']

+----------+----------+-----------+----------+----------------+---------+----------+---------+---------+------------+------------+------------+
|     mean |      std |       min |      25% |   50% (median) |     75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+==========+================+=========+==========+=========+=========+============+============+============+
| 0.463101 | 0.275391 | 0.0204074 | 0.224917 |       0.467412 | 0.69649 | 0.899437 | 13.4299 |      29 |  0.0758403 | -0.0233803 |   -1.30812 |
+----------+----------+-----------+----------+----------------+---------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.020407,0.049623,0.078839,0.108055,0.13727,0.166486,0.195702,0.224917,0.254133,0.28968,...,0.638563,0.667526,0.69649,0.725454,0.754418,0.786846,0.81496,0.843119,0.871278,0.899437


## 3.3* Tiền xử lý dữ liệu Nồng Độ CO2 của khu vực Đông Nam Á và Đông Á

### 3.3.1 Đông Nam Á

In [346]:
# Trích xuất tập dữ liệu

np_carbon_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    carbon = get_value_by_id_column(d_exdata['Carbon'], 'ISO3', iso3, col)
    dtrung = get_value_by_id_column(d_exdata['DienTichRung'], 'ISO3', iso3, col)
    np_carbon_dna[i, j] = carbon/dtrung

d_data_dna['NongDoCarbon'] = pd.DataFrame(np_carbon_dna, columns=l_timeline, dtype=np.double)
d_data_dna['NongDoCarbon'].insert(0, 'ISO3', ma_iso_dna)

in_vi_tri_gia_tri_thieu(d_data_dna['NongDoCarbon'], 'ISO3')
khai_quat_du_lieu(d_data_dna['NongDoCarbon'])


- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [347]:
d_data_dna['NongDoCarbon']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,BRN,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,...,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950,0.143950
1,IDN,0.095955,0.096847,0.097766,0.098714,0.099693,0.100703,0.101746,0.102825,0.103940,...,0.104324,0.104318,0.104312,0.104306,0.104300,0.104270,0.104300,0.104300,0.104300,0.104300
2,KHM,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,...,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810,0.050810
3,LAO,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,...,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530,0.068530
4,MMR,0.076622,0.076623,0.076624,0.076625,0.076626,0.076627,0.076628,0.076629,0.076630,...,0.075902,0.075843,0.075783,0.075722,0.075660,0.075620,0.075580,0.075560,0.075550,0.075530
5,MYS,0.123619,0.123126,0.122629,0.122128,0.121622,0.121111,0.120596,0.120075,0.119550,...,0.116020,0.117623,0.119209,0.120778,0.122330,0.122300,0.122300,0.122310,0.122310,0.122310
6,PHL,0.122055,0.122719,0.123392,0.124073,0.124763,0.125461,0.126168,0.126885,0.127610,...,0.134636,0.133861,0.133093,0.132333,0.131580,0.130830,0.130100,0.129370,0.128640,0.127920
7,SGP,0.059754,0.059789,0.059822,0.059855,0.059886,0.059917,0.059953,0.059983,0.060011,...,0.067629,0.069414,0.071259,0.073153,0.075112,0.076602,0.076598,0.076598,0.076603,0.076602
8,THA,0.082606,0.082614,0.082622,0.082630,0.082638,0.082646,0.082654,0.082662,0.082670,...,0.082994,0.083018,0.083042,0.083066,0.083090,0.083090,0.083080,0.083080,0.083070,0.083070
9,TLS,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,...,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240,0.090240


In [348]:
np_tong_ndcarbon_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['NongDoCarbon'], 'ISO3', iso3, col)
  np_tong_ndcarbon_dna[0, j] = sum
d_data_dna['TongNongDoCarbon'] = pd.DataFrame(np_tong_ndcarbon_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongNongDoCarbon'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongNongDoCarbon'], True)
d_data_dna['TongNongDoCarbon']

+----------+------------+---------+----------+----------------+----------+----------+---------+---------+-------------+------------+------------+
|     mean |        std |     min |      25% |   50% (median) |      75% |      max |     sum |   count |    variance |   skewness |   kurtosis |
+==========+============+=========+==========+================+==========+==========+=========+=========+=============+============+============+
| 0.969239 | 0.00943211 | 0.94966 | 0.965781 |         0.9682 | 0.977015 | 0.983142 | 28.1079 |      29 | 8.89647e-05 |  -0.223718 |  -0.480451 |
+----------+------------+---------+----------+----------------+----------+----------+---------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.94966,0.95202,0.954353,0.956662,0.958954,0.961234,0.963512,0.965781,0.968051,0.968023,...,0.971436,0.974203,0.977015,0.979864,0.982762,0.983142,0.982868,0.982168,0.981273,0.980562


In [349]:
# Chuẩn Hóa Min Max , min là 0.93 max là 1.0
d_data_dna['TongNongDoCarbon_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongNongDoCarbon'].iloc[0], 0.93, 1.0), columns=d_data_dna['TongNongDoCarbon'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongNongDoCarbon_ChuanHoa'], row_index=0)
d_data_dna['TongNongDoCarbon_ChuanHoa']

+----------+----------+----------+----------+----------------+---------+---------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |     75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+=========+=========+=========+=========+============+============+============+
| 0.560562 | 0.134744 | 0.280857 | 0.511162 |       0.545708 | 0.67165 | 0.75917 | 16.2563 |      29 |  0.0181561 |  -0.223718 |  -0.480451 |
+----------+----------+----------+----------+----------------+---------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.280857,0.314573,0.347896,0.380891,0.413632,0.446205,0.478748,0.511162,0.54359,0.543182,...,0.591941,0.631476,0.67165,0.712337,0.753742,0.75917,0.755262,0.745251,0.732474,0.722321


### 3.3.2 Đông Á

In [350]:
# Trích xuất tập dữ liệu

np_carbon_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    carbon = get_value_by_id_column(d_exdata['Carbon'], 'ISO3', iso3, col)
    dtrung = get_value_by_id_column(d_exdata['DienTichRung'], 'ISO3', iso3, col)
    np_carbon_donga[i, j] = carbon/dtrung

d_data_donga['NongDoCarbon'] = pd.DataFrame(np_carbon_donga, columns=l_timeline, dtype=np.double)
d_data_donga['NongDoCarbon'].insert(0, 'ISO3', ma_iso_donga)

in_vi_tri_gia_tri_thieu(d_data_donga['NongDoCarbon'], 'ISO3')
khai_quat_du_lieu(d_data_donga['NongDoCarbon'])


- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [351]:
np_tong_ndcarbon_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['NongDoCarbon'], 'ISO3', iso3, col)
  np_tong_ndcarbon_donga[0, j] = sum
d_data_donga['TongNongDoCarbon'] = pd.DataFrame(np_tong_ndcarbon_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongNongDoCarbon'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongNongDoCarbon'], True)
d_data_donga['TongNongDoCarbon']

+----------+-----------+----------+----------+----------------+----------+---------+---------+---------+-------------+------------+------------+
|     mean |       std |      min |      25% |   50% (median) |      75% |     max |     sum |   count |    variance |   skewness |   kurtosis |
+==========+===========+==========+==========+================+==========+=========+=========+=========+=============+============+============+
| 0.227893 | 0.0299921 | 0.181223 | 0.201579 |       0.227476 | 0.253368 | 0.27524 | 6.60889 |      29 | 0.000899524 |  0.0407429 |   -1.33096 |
+----------+-----------+----------+----------+----------------+----------+---------+---------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.181223,0.18413,0.187037,0.189944,0.192851,0.195759,0.198669,0.201579,0.20449,0.208308,...,0.246401,0.249886,0.253368,0.256846,0.26032,0.26412,0.26819,0.27029,0.27279,0.27524


## 3.4* Tiền xử lý dữ liệu Tân Suất Thiên Tai của khu vực Đông Nam Á và Đông Á

### 3.4.1 Đông Nam Á

In [352]:
np_thientai_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_thientai_dna[i, j] = get_value_by_id_column(d_exdata['TongThienTai'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatThienTai'] = pd.DataFrame(np_thientai_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatThienTai'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatThienTai'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [353]:
np_tong_tsthientai_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatThienTai'], 'ISO3', iso3, col)
  np_tong_tsthientai_dna[0, j] = sum
d_data_dna['TongTanSuatThienTai'] = pd.DataFrame(np_tong_tsthientai_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatThienTai'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatThienTai'], True)
d_data_dna['TongTanSuatThienTai']

+---------+----------+----------+---------+----------------+---------+---------+---------+---------+------------+------------+------------+
|    mean |      std |      min |     25% |   50% (median) |     75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+=========+==========+==========+=========+================+=========+=========+=========+=========+============+============+============+
| 1.30678 | 0.381581 | 0.551724 | 1.13793 |        1.31034 | 1.51724 | 2.17241 | 37.8966 |      29 |   0.145604 | -0.0647391 |    0.10707 |
+---------+----------+----------+---------+----------------+---------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.724138,0.827586,1.310345,0.965517,0.827586,0.586207,0.551724,1.275862,1.310345,1.344828,...,1.827586,1.275862,1.517241,1.241379,1.551724,1.586207,1.482759,1.137931,1.517241,2.172414


In [354]:
# Chuẩn Hóa Min Max , min là 0 max là 2.5
d_data_dna['TongTanSuatThienTai_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatThienTai'].iloc[0], 0.0, 2.5), columns=d_data_dna['TongTanSuatThienTai'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatThienTai_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatThienTai_ChuanHoa']

+----------+----------+---------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |     min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.522711 | 0.152633 | 0.22069 | 0.455172 |       0.524138 | 0.606897 | 0.868966 | 15.1586 |      29 |  0.0232967 | -0.0647391 |    0.10707 |
+----------+----------+---------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.289655,0.331034,0.524138,0.386207,0.331034,0.234483,0.22069,0.510345,0.524138,0.537931,...,0.731034,0.510345,0.606897,0.496552,0.62069,0.634483,0.593103,0.455172,0.606897,0.868966


### 3.4.2 Đông Á

In [355]:
np_thientai_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_thientai_donga[i, j] = get_value_by_id_column(d_exdata['TongThienTai'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatThienTai'] = pd.DataFrame(np_thientai_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatThienTai'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatThienTai'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [356]:
np_tong_tsthientai_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatThienTai'], 'ISO3', iso3, col)
  np_tong_tsthientai_donga[0, j] = sum
d_data_donga['TongTanSuatThienTai'] = pd.DataFrame(np_tong_tsthientai_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatThienTai'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatThienTai'], True)
d_data_donga['TongTanSuatThienTai']

+---------+----------+---------+----------+----------------+---------+---------+---------+---------+------------+------------+------------+
|    mean |      std |     min |      25% |   50% (median) |     75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+=========+==========+=========+==========+================+=========+=========+=========+=========+============+============+============+
| 1.06421 | 0.285801 | 0.62069 | 0.827586 |        1.06897 | 1.31034 | 1.62069 | 30.8621 |      29 |   0.081682 |   0.292268 |   -1.09322 |
+---------+----------+---------+----------+----------------+---------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.689655,0.896552,0.689655,0.62069,0.793103,0.827586,0.793103,0.827586,1.275862,1.344828,...,0.827586,1.206897,1.517241,1.62069,1.517241,1.448276,1.103448,1.103448,1.068966,0.862069


In [357]:
# Chuẩn Hóa Min Max , min là 0.54 max là 1.71
d_data_donga['TongTanSuatThienTai_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongTanSuatThienTai'].iloc[0], 0.54, 1.71), columns=d_data_donga['TongTanSuatThienTai'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatThienTai_ChuanHoa'], row_index=0)
d_data_donga['TongTanSuatThienTai_ChuanHoa']

+----------+----------+-----------+--------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |       min |    25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+========+================+==========+==========+=========+=========+============+============+============+
| 0.448042 | 0.244274 | 0.0689655 | 0.2458 |       0.452107 | 0.658414 | 0.923666 | 12.9932 |      29 |  0.0596698 |   0.292268 |   -1.09322 |
+----------+----------+-----------+--------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.12791,0.304745,0.12791,0.068966,0.216328,0.2458,0.216328,0.2458,0.628942,0.687887,...,0.2458,0.569997,0.835249,0.923666,0.835249,0.776304,0.48158,0.48158,0.452107,0.275273


## 3.5 Tiền xử lý dữ liệu Diện Tích Đất của khu vực Đông Nam Á và Đông Á

### 3.5.1 Đông Nam Á

In [358]:
# Trích xuất tập dữ liệu

np_dtdat_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_dtdat_dna[i, j] = get_value_by_id_column(d_exdata['DienTichDat'], 'ISO3', iso3, col)

d_data_dna['DienTichDat'] = pd.DataFrame(np_dtdat_dna, columns=l_timeline, dtype=np.double)
d_data_dna['DienTichDat'].insert(0, 'ISO3', ma_iso_dna)

in_vi_tri_gia_tri_thieu(d_data_dna['DienTichDat'], 'ISO3')
khai_quat_du_lieu(d_data_dna['DienTichDat'])


- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+==

In [359]:
d_data_dna['DienTichDat']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,BRN,527.0,527.0,527.0,527.0,527.0,527.0,527.0,527.0,527.0,...,527.0,527.0,527.000,527.0,527.0,527.0,527.0,527.0,527.0,527.0
1,IDN,181157.0,181157.0,181157.0,181157.0,181157.0,181157.0,181157.0,181157.0,187751.9,...,187751.9,187751.9,187751.900,187751.9,187751.9,187751.9,187751.9,187751.9,187751.9,187751.9
2,KHM,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0,...,17652.0,17652.0,17652.000,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0,17652.0
3,LAO,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0,...,23080.0,23080.0,23080.000,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0,23080.0
4,MMR,65354.0,65354.0,65354.0,65354.0,65354.0,65354.0,65354.0,65354.0,65354.0,...,65329.0,65315.0,65308.000,65308.0,65308.0,65278.0,65275.0,65270.0,65270.0,65267.0
5,MYS,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0,...,32855.0,32855.0,32855.000,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0,32855.0
6,PHL,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0,...,29817.0,29817.0,29817.000,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0,29817.0
7,SGP,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,70.4,70.6,70.700,70.9,70.9,70.9,71.2,71.4,71.6,71.8
8,THA,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0,...,51089.0,51089.0,51089.000,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0,51089.0
9,TLS,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0,...,1487.0,1487.0,1487.000,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0,1487.0


In [360]:
np_tong_ddat_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['DienTichDat'], 'ISO3', iso3, col)
  np_tong_ddat_dna[0, j] = sum
d_data_dna['TongDienTichDat'] = pd.DataFrame(np_tong_ddat_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongDienTichDat'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongDienTichDat'], True)
d_data_dna['TongDienTichDat']

+--------+---------+--------+--------+----------------+--------+--------+------------+---------+-------------+------------+------------+
|   mean |     std |    min |    25% |   50% (median) |    75% |    max |        sum |   count |    variance |   skewness |   kurtosis |
+========+=========+========+========+================+========+========+============+=========+=============+============+============+
| 439449 | 2398.05 | 435634 | 435634 |         440936 | 440947 | 440990 | 1.2744e+07 |      29 | 5.75062e+06 |   -1.05417 |  -0.952919 |
+--------+---------+--------+--------+----------------+--------+--------+------------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,435634.0,435634.0,435634.0,435634.0,435634.0,435634.0,435634.0,435634.0,440785.9,440788.9,...,440986.7,440990.2,440938.333,440974.9,440976.1,440946.5,440945.4,440943.2,440943.4,440940.6


In [361]:
# Chuẩn Hóa Min Max , min là 433900, max là 442900
d_data_dna['TongDienTichDat_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongDienTichDat'].iloc[0], 433900, 442900), columns=d_data_dna['TongDienTichDat'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongDienTichDat_ChuanHoa'], row_index=0)
d_data_dna['TongDienTichDat_ChuanHoa']

+----------+---------+----------+----------+----------------+----------+--------+---------+---------+------------+------------+------------+
|     mean |     std |      min |      25% |   50% (median) |      75% |    max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+=========+==========+==========+================+==========+========+=========+=========+============+============+============+
| 0.616591 | 0.26645 | 0.192667 | 0.192667 |         0.7818 | 0.782944 | 0.7878 | 17.8811 |      29 |  0.0709954 |   -1.05417 |  -0.952919 |
+----------+---------+----------+----------+----------------+----------+--------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.192667,0.192667,0.192667,0.192667,0.192667,0.192667,0.192667,0.192667,0.7651,0.765433,...,0.787411,0.7878,0.782037,0.7861,0.786233,0.782944,0.782822,0.782578,0.7826,0.782289


### 3.5.2 Đông Á

In [362]:
# Trích xuất tập dữ liệu

np_dtdat_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_dtdat_donga[i, j] = get_value_by_id_column(d_exdata['DienTichDat'], 'ISO3', iso3, col)

d_data_donga['DienTichDat'] = pd.DataFrame(np_dtdat_donga, columns=l_timeline, dtype=np.double)
d_data_donga['DienTichDat'].insert(0, 'ISO3', ma_iso_donga)

in_vi_tri_gia_tri_thieu(d_data_donga['DienTichDat'], 'ISO3')
khai_quat_du_lieu(d_data_donga['DienTichDat'])


- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [363]:
d_data_donga['DienTichDat']

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,CHN,938825.0000,938825.0000,938825.0000,938824.0000,938823.0000,938823.0000,938823.0000,938823.0000,938822.0000,...,938821.1000,938821.1000,938821.1000,938821.1000,938821.10,938821.00,938821.00,938821.0,938821.0,938821.0000
1,JPN,36460.0000,36460.0000,36460.0000,36460.0000,36450.0000,36450.0000,36450.0000,36450.0000,36450.0000,...,36450.0000,36450.0000,36450.0000,36450.0000,36450.00,36450.00,36450.00,36450.0,36450.0,36450.0000
2,KOR,9646.0000,9646.0000,9646.0000,9646.0000,9646.0000,9646.0000,9646.0000,9646.0000,9646.0000,...,9719.0000,9731.0000,9742.6000,9743.0000,9744.50,9748.90,9751.00,9752.0,9760.0,9760.0000
3,MNG,154743.8742,154743.8742,154743.8742,154743.8742,154743.8742,154743.8742,154743.8742,154743.8742,154743.8742,...,155725.1947,155725.1946,155725.3053,155725.2921,155725.54,155725.54,155725.54,155725.5,155725.8,155750.6803
4,PRK,12041.0000,12041.0000,12041.0000,12041.0000,12041.0000,12041.0000,12041.0000,12041.0000,12041.0000,...,12041.0000,12041.0000,12041.0000,12041.0000,12041.00,12041.00,12041.00,12041.0,12041.0,12041.0000


In [364]:
np_tong_ddat_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['DienTichDat'], 'ISO3', iso3, col)
  np_tong_ddat_donga[0, j] = sum
d_data_donga['TongDienTichDat'] = pd.DataFrame(np_tong_ddat_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongDienTichDat'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongDienTichDat'], True)
d_data_donga['TongDienTichDat']

+-------------+---------+------------+-------------+----------------+-------------+-------------+-------------+---------+------------+------------+------------+
|        mean |     std |        min |         25% |   50% (median) |         75% |         max |         sum |   count |   variance |   skewness |   kurtosis |
+=============+=========+============+=============+================+=============+=============+=============+=========+============+============+============+
| 1.15234e+06 | 507.307 | 1.1517e+06 | 1.15172e+06 |    1.15275e+06 | 1.15278e+06 | 1.15282e+06 | 3.34178e+07 |      29 |     257361 |   -0.41995 |   -1.85486 |
+-------------+---------+------------+-------------+----------------+-------------+-------------+-------------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1.151716e+06,1.151716e+06,1.151716e+06,1.151715e+06,1.151704e+06,1.151704e+06,1.151704e+06,1.151704e+06,1.151703e+06,1.151730e+06,...,1.152756e+06,1.152768e+06,1.152780e+06,1.152780e+06,1152782.14,1152786.44,1152788.54,1152789.5,1152797.8,1.152823e+06


In [365]:
# Chuẩn Hóa Min Max , min là 1151630, max là 1152828
d_data_donga['TongDienTichDat_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongDienTichDat'].iloc[0], 1151630 , 1152828), columns=d_data_donga['TongDienTichDat'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongDienTichDat_ChuanHoa'], row_index=0)
d_data_donga['TongDienTichDat_ChuanHoa']

+----------+----------+-----------+-----------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |       min |       25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+===========+================+==========+=========+=========+=========+============+============+============+
| 0.591479 | 0.423462 | 0.0608299 | 0.0716813 |       0.931594 | 0.959938 | 0.99556 | 17.1529 |      29 |    0.17932 |   -0.41995 |   -1.85486 |
+----------+----------+-----------+-----------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.071681,0.071681,0.071681,0.070847,0.061665,0.061665,0.061665,0.061665,0.06083,0.083718,...,0.940146,0.950162,0.959938,0.960261,0.96172,0.965309,0.967062,0.967863,0.974791,0.99556


## 3.6 Tiền Xử lý dữ liệu Mực Nước Biển các nước Đông Nam Á và Đông Á

### 3.6.1 Đông Nam Á

In [366]:
np_tb_bien_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(vung_bien_tiep_giap_dong_nam_a):
    sum += get_value_by_id_column(d_exdata['MucNuoc'], 'ISO3', iso3, col)
  np_tb_bien_dna[0, j] = sum/len(vung_bien_tiep_giap_dong_nam_a)
d_data_dna['TrungBinhMucNuoc'] = pd.DataFrame(np_tb_bien_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TrungBinhMucNuoc'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TrungBinhMucNuoc'], True)
d_data_dna['TrungBinhMucNuoc']

+---------+---------+---------+-----------+----------------+---------+---------+---------+---------+------------+------------+------------+
|    mean |     std |     min |       25% |   50% (median) |     75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+=========+=========+=========+===========+================+=========+=========+=========+=========+============+============+============+
| 24.8782 | 37.9417 | -48.265 | -0.744642 |        27.5674 | 60.2276 | 83.3058 | 721.469 |      29 |    1439.57 |  -0.158325 |  -0.952823 |
+---------+---------+---------+-----------+----------------+---------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,-10.1525,-34.034118,-42.159853,-11.581079,7.246319,-48.265,-9.701364,23.315303,35.086471,27.567401,...,60.337842,65.754783,81.422343,44.026111,32.728851,77.578857,67.455539,56.103986,32.114122,83.305815


In [367]:
# Chuẩn Hóa Min Max , min là -54, max là 106
d_data_dna['TrungBinhMucNuoc_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TrungBinhMucNuoc'].iloc[0], -54, 106), columns=d_data_dna['TrungBinhMucNuoc'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TrungBinhMucNuoc_ChuanHoa'], row_index=0)
d_data_dna['TrungBinhMucNuoc_ChuanHoa']

+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |       min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.492989 | 0.237135 | 0.0358437 | 0.332846 |       0.509796 | 0.713922 | 0.858161 | 14.2967 |      29 |  0.0562332 |  -0.158325 |  -0.952823 |
+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.274047,0.124787,0.074001,0.265118,0.382789,0.035844,0.276866,0.483221,0.55679,0.509796,...,0.714612,0.748467,0.84639,0.612663,0.542055,0.822368,0.759097,0.68815,0.538213,0.858161


### 3.6.2 Đông Á

In [368]:
np_tb_bien_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(vung_bien_tiep_giap_dong_a):
    sum += get_value_by_id_column(d_exdata['MucNuoc'], 'ISO3', iso3, col)
  np_tb_bien_donga[0, j] = sum/len(vung_bien_tiep_giap_dong_a)
d_data_donga['TrungBinhMucNuoc'] = pd.DataFrame(np_tb_bien_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TrungBinhMucNuoc'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TrungBinhMucNuoc'], True)
d_data_donga['TrungBinhMucNuoc']

+---------+---------+--------+---------+----------------+---------+---------+-------+---------+------------+------------+------------+
|    mean |     std |    min |     25% |   50% (median) |     75% |     max |   sum |   count |   variance |   skewness |   kurtosis |
+=========+=========+========+=========+================+=========+=========+=======+=========+============+============+============+
| 21.4655 | 31.1253 | -38.93 | 1.19273 |        20.0018 | 40.3042 | 80.3455 | 622.5 |      29 |    968.786 |   -0.17686 |  -0.557567 |
+---------+---------+--------+---------+----------------+---------+---------+-------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,-38.93,-34.90451,-15.722157,-23.546297,-17.544023,-10.500202,1.192727,20.001818,15.596471,20.772751,...,27.029087,60.304851,33.432779,47.582558,34.909459,67.720048,54.576078,48.249009,65.603063,80.345495


In [369]:
# Chuẩn Hóa Min Max , min là -45, max là 81
d_data_donga['TrungBinhMucNuoc_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TrungBinhMucNuoc'].iloc[0], -45, 81), columns=d_data_donga['TrungBinhMucNuoc'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TrungBinhMucNuoc_ChuanHoa'], row_index=0)
d_data_donga['TrungBinhMucNuoc_ChuanHoa']

+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |       min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.527504 | 0.247026 | 0.0481746 | 0.366609 |       0.515887 | 0.677018 | 0.994806 | 15.2976 |      29 |  0.0610221 |   -0.17686 |  -0.557567 |
+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.048175,0.080123,0.232364,0.170267,0.217905,0.273808,0.366609,0.515887,0.480924,0.522006,...,0.571659,0.835753,0.622482,0.734782,0.634202,0.894604,0.790286,0.740072,0.877802,0.994806


## 3.7* Tiền Xử lý dữ liệu tần suất Hạn Hán của khu vực Đông Nam Á và Đông Á

### 3.7.1 Đông Nam Á

In [370]:
np_HanHan_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_HanHan_dna[i, j] = get_value_by_id_column(d_exdata['HanHan'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatHanHan'] = pd.DataFrame(np_HanHan_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatHanHan'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatHanHan'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [371]:
np_tong_tsHanHan_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatHanHan'], 'ISO3', iso3, col)
  np_tong_tsHanHan_dna[0, j] = sum
d_data_dna['TongTanSuatHanHan'] = pd.DataFrame(np_tong_tsHanHan_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatHanHan'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatHanHan'], True)
d_data_dna['TongTanSuatHanHan']

+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+
|      mean |       std |   min |   25% |   50% (median) |       75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+===========+===========+=======+=======+================+===========+==========+=========+=========+============+============+============+
| 0.0428062 | 0.0457606 |     0 |     0 |      0.0344828 | 0.0689655 | 0.137931 | 1.24138 |      29 | 0.00209404 |    0.90086 |  -0.201156 |
+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.034483,0.034483,0.0,0.0,0.068966,0.068966,0.103448,0.0,0.034483,...,0.034483,0.034483,0.0,0.068966,0.137931,0.068966,0.0,0.0,0.137931,0.0


In [372]:
# Chuẩn Hóa Min Max , min là 0.0, max là 0.16
d_data_dna['TongTanSuatHanHan_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatHanHan'].iloc[0], 0.0, 0.16), columns=d_data_dna['TongTanSuatHanHan'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatHanHan_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatHanHan_ChuanHoa']

+----------+----------+-------+-------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |   min |   25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=======+=======+================+==========+==========+=========+=========+============+============+============+
| 0.267539 | 0.286004 |     0 |     0 |       0.215517 | 0.431034 | 0.862069 | 7.75862 |      29 |  0.0817983 |    0.90086 |  -0.201156 |
+----------+----------+-------+-------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.215517,0.215517,0.0,0.0,0.431034,0.431034,0.646552,0.0,0.215517,...,0.215517,0.215517,0.0,0.431034,0.862069,0.431034,0.0,0.0,0.862069,0.0


### 3.7.2 Đông Á

In [373]:
np_HanHan_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_HanHan_donga[i, j] = get_value_by_id_column(d_exdata['HanHan'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatHanHan'] = pd.DataFrame(np_HanHan_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatHanHan'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatHanHan'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [374]:
np_tong_tsHanHan_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatHanHan'], 'ISO3', iso3, col)
  np_tong_tsHanHan_donga[0, j] = sum
d_data_donga['TongTanSuatHanHan'] = pd.DataFrame(np_tong_tsHanHan_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatHanHan'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatHanHan'], True)
d_data_donga['TongTanSuatHanHan']

+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+
|      mean |       std |   min |   25% |   50% (median) |       75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+===========+===========+=======+=======+================+===========+==========+=========+=========+============+============+============+
| 0.0428062 | 0.0364642 |     0 |     0 |      0.0344828 | 0.0689655 | 0.137931 | 1.24138 |      29 | 0.00132964 |   0.646178 |   0.144079 |
+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.068966,0.0,0.068966,0.034483,0.0,0.034483,0.0,0.034483,0.137931,0.103448,...,0.0,0.034483,0.034483,0.068966,0.068966,0.034483,0.068966,0.0,0.034483,0.0


In [375]:
# Chuẩn Hóa Min Max , min là 0.0, max là 0.07
d_data_donga['TongTanSuatHanHan_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongTanSuatHanHan'].iloc[0], 0.0, 0.07), columns=d_data_donga['TongTanSuatHanHan'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatHanHan_ChuanHoa'], row_index=0)
d_data_donga['TongTanSuatHanHan_ChuanHoa']

+----------+----------+-------+-------+----------------+----------+---------+--------+---------+------------+------------+------------+
|     mean |      std |   min |   25% |   50% (median) |      75% |     max |    sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=======+=======+================+==========+=========+========+=========+============+============+============+
| 0.611517 | 0.520917 |     0 |     0 |       0.492611 | 0.985222 | 1.97044 | 17.734 |      29 |   0.271355 |   0.646178 |   0.144079 |
+----------+----------+-------+-------+----------------+----------+---------+--------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.985222,0.0,0.985222,0.492611,0.0,0.492611,0.0,0.492611,1.970443,1.477833,...,0.0,0.492611,0.492611,0.985222,0.985222,0.492611,0.985222,0.0,0.492611,0.0


## 3.8* Tiền Xử lý dữ liệu tần suất Lũ Lụt của khu vực Đông Nam Á và Đông Á

### 3.8.1 Đông Nam Á

In [376]:
np_lulut_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_lulut_dna[i, j] = get_value_by_id_column(d_exdata['LuLut'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatLuLut'] = pd.DataFrame(np_lulut_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatLuLut'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatLuLut'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [377]:
np_tong_tsLuLut_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatLuLut'], 'ISO3', iso3, col)
  np_tong_tsLuLut_dna[0, j] = sum
d_data_dna['TongTanSuatLuLut'] = pd.DataFrame(np_tong_tsLuLut_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatLuLut'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatLuLut'], True)
d_data_dna['TongTanSuatLuLut']

+----------+----------+----------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+=========+=========+=========+============+============+============+
| 0.678954 | 0.286006 | 0.103448 | 0.517241 |       0.655172 | 0.896552 | 1.24138 | 19.6897 |      29 |  0.0817992 |  -0.096593 |  -0.228777 |
+----------+----------+----------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.275862,0.344828,0.551724,0.551724,0.413793,0.103448,0.103448,0.62069,0.517241,0.827586,...,1.068966,0.655172,1.0,0.62069,0.689655,0.896552,0.793103,0.586207,0.896552,1.241379


In [378]:
d_data_dna['TongTanSuatLuLut_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatLuLut'].iloc[0], 0.0, 1.44), columns=d_data_dna['TongTanSuatLuLut'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatLuLut_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatLuLut_ChuanHoa']

+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |       min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+===========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.471496 | 0.198615 | 0.0718391 | 0.359195 |       0.454981 | 0.622605 | 0.862069 | 13.6734 |      29 |  0.0394479 |  -0.096593 |  -0.228777 |
+----------+----------+-----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.191571,0.239464,0.383142,0.383142,0.287356,0.071839,0.071839,0.431034,0.359195,0.574713,...,0.742337,0.454981,0.694444,0.431034,0.478927,0.622605,0.550766,0.407088,0.622605,0.862069


### 3.8.2 Đông Á

In [379]:
np_lulut_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_lulut_donga[i, j] = get_value_by_id_column(d_exdata['LuLut'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatLuLut'] = pd.DataFrame(np_lulut_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatLuLut'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatLuLut'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [380]:
np_tong_tsLuLut_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatLuLut'], 'ISO3', iso3, col)
  np_tong_tsLuLut_donga[0, j] = sum
d_data_donga['TongTanSuatLuLut'] = pd.DataFrame(np_tong_tsLuLut_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatLuLut'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatLuLut'], True)
d_data_donga['TongTanSuatLuLut']

+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.388823 | 0.140945 | 0.172414 | 0.275862 |        0.37931 | 0.482759 | 0.862069 | 11.2759 |      29 |  0.0198655 |    1.19215 |    3.07983 |
+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.241379,0.172414,0.241379,0.206897,0.241379,0.310345,0.275862,0.344828,0.413793,0.482759,...,0.344828,0.482759,0.551724,0.482759,0.482759,0.517241,0.448276,0.37931,0.344828,0.413793


In [381]:
d_data_donga['TongTanSuatLuLut_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongTanSuatLuLut'].iloc[0], 0.171, 0.567), columns=d_data_donga['TongTanSuatLuLut'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatLuLut_ChuanHoa'], row_index=0)
d_data_donga['TongTanSuatLuLut_ChuanHoa']

+----------+----------+------------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |        min |      25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+============+==========+================+==========+=========+=========+=========+============+============+============+
| 0.550058 | 0.355922 | 0.00357018 | 0.264803 |       0.526036 | 0.787269 | 1.74512 | 15.9517 |      29 |    0.12668 |    1.19215 |    3.07983 |
+----------+----------+------------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.177726,0.00357,0.177726,0.090648,0.177726,0.351881,0.264803,0.438959,0.613114,0.787269,...,0.438959,0.787269,0.961425,0.787269,0.787269,0.874347,0.700192,0.526036,0.438959,0.613114


## 3.9* Tiền Xử lý dữ liệu tần suất Bão của khu vực Đông Nam Á và Đông Á

### 3.9.1 Đông Nam Á

In [382]:
np_Bao_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_Bao_dna[i, j] = get_value_by_id_column(d_exdata['Bao'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatBao'] = pd.DataFrame(np_Bao_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatBao'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatBao'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [383]:
np_tong_tsBao_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatBao'], 'ISO3', iso3, col)
  np_tong_tsBao_dna[0, j] = sum
d_data_dna['TongTanSuatBao'] = pd.DataFrame(np_tong_tsBao_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatBao'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatBao'], True)
d_data_dna['TongTanSuatBao']

+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.444709 | 0.146977 | 0.241379 | 0.344828 |       0.413793 | 0.517241 | 0.827586 | 12.8966 |      29 |  0.0216022 |   0.709823 |  0.0548524 |
+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.37931,0.413793,0.551724,0.344828,0.275862,0.344828,0.241379,0.310345,0.517241,0.344828,...,0.517241,0.482759,0.413793,0.448276,0.517241,0.482759,0.62069,0.448276,0.344828,0.827586


In [384]:
d_data_dna['TongTanSuatBao_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatBao'].iloc[0], 0.0, 1.06), columns=d_data_dna['TongTanSuatBao'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatBao_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatBao_ChuanHoa']

+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+==========+==========+=========+=========+============+============+============+
| 0.419536 | 0.138658 | 0.227716 | 0.325309 |       0.390371 | 0.487964 | 0.780742 | 12.1666 |      29 |  0.0192259 |   0.709823 |  0.0548524 |
+----------+----------+----------+----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.35784,0.390371,0.520494,0.325309,0.260247,0.325309,0.227716,0.292778,0.487964,0.325309,...,0.487964,0.455433,0.390371,0.422902,0.487964,0.455433,0.585556,0.422902,0.325309,0.780742


### 3.9.2 Đông Á

In [385]:
np_Bao_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_Bao_donga[i, j] = get_value_by_id_column(d_exdata['Bao'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatBao'] = pd.DataFrame(np_Bao_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatBao'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatBao'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [386]:
np_tong_tsBao_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatBao'], 'ISO3', iso3, col)
  np_tong_tsBao_donga[0, j] = sum
d_data_donga['TongTanSuatBao'] = pd.DataFrame(np_tong_tsBao_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatBao'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatBao'], True)
d_data_donga['TongTanSuatBao']

+----------+----------+----------+----------+----------------+---------+----------+---------+---------+------------+------------+------------+
|     mean |      std |      min |      25% |   50% (median) |     75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+==========+==========+================+=========+==========+=========+=========+============+============+============+
| 0.483948 | 0.176427 | 0.275862 | 0.344828 |       0.413793 | 0.62069 | 0.931034 | 14.0345 |      29 |  0.0311264 |   0.750157 |   -0.19305 |
+----------+----------+----------+----------+----------------+---------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.37931,0.586207,0.310345,0.310345,0.310345,0.275862,0.448276,0.37931,0.62069,0.551724,...,0.310345,0.62069,0.758621,0.931034,0.793103,0.62069,0.413793,0.586207,0.448276,0.37931


In [387]:
d_data_donga['TongTanSuatBao_ChuanHoa'] =  d_data_donga['TongTanSuatBao']

## 3.10* Tiền Xử lý dữ liệu tần suất Sạt Lỡ của khu vực Đông Nam Á và Đông Á

### 3.10.1 Đông Nam Á

In [388]:
np_SatLo_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_SatLo_dna[i, j] = get_value_by_id_column(d_exdata['SatLo'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatSatLo'] = pd.DataFrame(np_SatLo_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatSatLo'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatSatLo'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [389]:
np_tong_tsSatLo_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatSatLo'], 'ISO3', iso3, col)
  np_tong_tsSatLo_dna[0, j] = sum
d_data_dna['TongTanSuatSatLo'] = pd.DataFrame(np_tong_tsSatLo_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatSatLo'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatSatLo'], True)
d_data_dna['TongTanSuatSatLo']

+----------+-----------+-------+-----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|     mean |       std |   min |       25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+===========+=======+===========+================+==========+==========+=========+=========+============+============+============+
| 0.116528 | 0.0654354 |     0 | 0.0689655 |       0.103448 | 0.172414 | 0.241379 | 3.37931 |      29 | 0.00428179 |   0.250831 |   -0.45056 |
+----------+-----------+-------+-----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.068966,0.034483,0.137931,0.034483,0.137931,0.0,0.0,0.206897,0.241379,0.137931,...,0.206897,0.103448,0.103448,0.068966,0.172414,0.103448,0.068966,0.103448,0.103448,0.103448


In [390]:
d_data_dna['TongTanSuatSatLo_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatSatLo'].iloc[0], 0.0, 0.27), columns=d_data_dna['TongTanSuatSatLo'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatSatLo_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatSatLo_ChuanHoa']

+----------+----------+-------+----------+----------------+---------+----------+--------+---------+------------+------------+------------+
|     mean |      std |   min |      25% |   50% (median) |     75% |      max |    sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=======+==========+================+=========+==========+========+=========+============+============+============+
| 0.431585 | 0.242353 |     0 | 0.255428 |       0.383142 | 0.63857 | 0.893997 | 12.516 |      29 |  0.0587351 |   0.250831 |   -0.45056 |
+----------+----------+-------+----------+----------------+---------+----------+--------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.255428,0.127714,0.510856,0.127714,0.510856,0.0,0.0,0.766284,0.893997,0.510856,...,0.766284,0.383142,0.383142,0.255428,0.63857,0.383142,0.255428,0.383142,0.383142,0.383142


### 3.10.2 Đông Á

In [391]:
np_SatLo_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_SatLo_donga[i, j] = get_value_by_id_column(d_exdata['SatLo'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatSatLo'] = pd.DataFrame(np_SatLo_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatSatLo'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatSatLo'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [392]:
np_tong_tsSatLo_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatSatLo'], 'ISO3', iso3, col)
  np_tong_tsSatLo_donga[0, j] = sum
d_data_donga['TongTanSuatSatLo'] = pd.DataFrame(np_tong_tsSatLo_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatSatLo'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatSatLo'], True)
d_data_donga['TongTanSuatSatLo']

+-----------+-----------+-------+-----------+----------------+----------+----------+---------+---------+------------+------------+------------+
|      mean |       std |   min |       25% |   50% (median) |      75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+===========+===========+=======+===========+================+==========+==========+=========+=========+============+============+============+
| 0.0903686 | 0.0739652 |     0 | 0.0344828 |      0.0689655 | 0.137931 | 0.275862 | 2.62069 |      29 | 0.00547085 |   0.952981 |   0.231581 |
+-----------+-----------+-------+-----------+----------------+----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.137931,0.034483,0.034483,0.206897,0.103448,0.068966,0.034483,0.068966,0.206897,...,0.103448,0.034483,0.103448,0.103448,0.034483,0.103448,0.172414,0.0,0.068966,0.034483


In [393]:
d_data_donga['TongTanSuatSatLo_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongTanSuatSatLo'].iloc[0], 0.0, 0.19), columns=d_data_donga['TongTanSuatSatLo'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatSatLo_ChuanHoa'], row_index=0)
d_data_donga['TongTanSuatSatLo_ChuanHoa']

+----------+----------+-------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |   min |      25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=======+==========+================+==========+=========+=========+=========+============+============+============+
| 0.475624 | 0.389291 |     0 | 0.181488 |       0.362976 | 0.725953 | 1.45191 | 13.7931 |      29 |   0.151547 |   0.952981 |   0.231581 |
+----------+----------+-------+----------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.725953,0.181488,0.181488,1.088929,0.544465,0.362976,0.181488,0.362976,1.088929,...,0.544465,0.181488,0.544465,0.544465,0.181488,0.544465,0.907441,0.0,0.362976,0.181488


## 3.11* Tiền Xử lý dữ liệu tần suất Nhiệt Độ Cực Đoan của khu vực Đông Nam Á và Đông Á

### 3.11.1 Đông Nam Á

In [394]:
np_NhietDoCucDoan_dna = np.full((len(ma_iso_dna), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_dna):
  for j, col in enumerate(l_timeline):
    np_NhietDoCucDoan_dna[i, j] = get_value_by_id_column(d_exdata['NhietDoCucDoan'], 'ISO3', iso3, col)/len(l_timeline)

d_data_dna['TanSuatNhietDoCucDoan'] = pd.DataFrame(np_NhietDoCucDoan_dna, columns=l_timeline, dtype=np.double)
d_data_dna['TanSuatNhietDoCucDoan'].insert(0, 'ISO3', ma_iso_dna)

khai_quat_du_lieu(d_data_dna['TanSuatNhietDoCucDoan'])

- Kích thước của dữ liệu: (11, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+===

In [395]:
np_tong_tsNhietDoCucDoan_dna = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_dna):
    sum += get_value_by_id_column(d_data_dna['TanSuatNhietDoCucDoan'], 'ISO3', iso3, col)
  np_tong_tsNhietDoCucDoan_dna[0, j] = sum
d_data_dna['TongTanSuatNhietDoCucDoan'] = pd.DataFrame(np_tong_tsNhietDoCucDoan_dna, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatNhietDoCucDoan'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_dna['TongTanSuatNhietDoCucDoan'], True)
d_data_dna['TongTanSuatNhietDoCucDoan']

+------------+------------+-------+-------+----------------+-------+-----------+-----------+---------+-------------+------------+------------+
|       mean |        std |   min |   25% |   50% (median) |   75% |       max |       sum |   count |    variance |   skewness |   kurtosis |
+============+============+=======+=======+================+=======+===========+===========+=========+=============+============+============+
| 0.00237812 | 0.00889244 |     0 |     0 |              0 |     0 | 0.0344828 | 0.0689655 |      29 | 7.90755e-05 |    3.59052 |    11.6955 |
+------------+------------+-------+-------+----------------+-------+-----------+-----------+---------+-------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.0,0.0


In [396]:
d_data_dna['TongTanSuatNhietDoCucDoan_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_dna['TongTanSuatNhietDoCucDoan'].iloc[0], 0.0, 0.043), columns=d_data_dna['TongTanSuatNhietDoCucDoan'].columns)
in_bang_thong_ke_mo_ta(d_data_dna['TongTanSuatNhietDoCucDoan_ChuanHoa'], row_index=0)
d_data_dna['TongTanSuatNhietDoCucDoan_ChuanHoa']

+-----------+----------+-------+-------+----------------+-------+----------+---------+---------+------------+------------+------------+
|      mean |      std |   min |   25% |   50% (median) |   75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+===========+==========+=======+=======+================+=======+==========+=========+=========+============+============+============+
| 0.0553051 | 0.206801 |     0 |     0 |              0 |     0 | 0.801925 | 1.60385 |      29 |  0.0427666 |    3.59052 |    11.6955 |
+-----------+----------+-------+-------+----------------+-------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.801925,0.0,0.801925,0.0,0.0,0.0,0.0


### 3.11.2 Đông Á

In [397]:
np_NhietDoCucDoan_donga = np.full((len(ma_iso_donga), len(l_timeline)), np.nan)

for i, iso3 in enumerate(ma_iso_donga):
  for j, col in enumerate(l_timeline):
    np_NhietDoCucDoan_donga[i, j] = get_value_by_id_column(d_exdata['NhietDoCucDoan'], 'ISO3', iso3, col)/len(l_timeline)

d_data_donga['TanSuatNhietDoCucDoan'] = pd.DataFrame(np_NhietDoCucDoan_donga, columns=l_timeline, dtype=np.double)
d_data_donga['TanSuatNhietDoCucDoan'].insert(0, 'ISO3', ma_iso_donga)

khai_quat_du_lieu(d_data_donga['TanSuatNhietDoCucDoan'])

- Kích thước của dữ liệu: (5, 30)

- Tên các cột: ['ISO3', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

- Kiểu dữ liệu của các cột: 
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|  ISO3   |  1992   |  1993   |  1994   |  1995   |  1996   |  1997   |  1998   |  1999   |  2000   |  2001   |  2002   |  2003   |  2004   |  2005   |  2006   |  2007   |  2008   |  2009   |  2010   |  2011   |  2012   |  2013   |  2014   |  2015   |  2016   |  2017   |  2018   |  2019   |  2020   |
+=========+=========+=========+=========+=========+=========+=========+====

In [398]:
np_tong_tsNhietDoCucDoan_donga = np.full((1, len(l_timeline)), np.nan)
for j, col in enumerate(l_timeline):
  sum = 0.0
  for i, iso3 in enumerate(ma_iso_donga):
    sum += get_value_by_id_column(d_data_donga['TanSuatNhietDoCucDoan'], 'ISO3', iso3, col)
  np_tong_tsNhietDoCucDoan_donga[0, j] = sum
d_data_donga['TongTanSuatNhietDoCucDoan'] = pd.DataFrame(np_tong_tsNhietDoCucDoan_donga, columns=l_timeline, dtype=np.double)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatNhietDoCucDoan'], row_index=0)
ve_bieu_do_thong_ke_mo_ta(d_data_donga['TongTanSuatNhietDoCucDoan'], True)
d_data_donga['TongTanSuatNhietDoCucDoan']

+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+
|      mean |       std |   min |   25% |   50% (median) |       75% |      max |     sum |   count |   variance |   skewness |   kurtosis |
+===========+===========+=======+=======+================+===========+==========+=========+=========+============+============+============+
| 0.0439952 | 0.0503905 |     0 |     0 |      0.0344828 | 0.0689655 | 0.172414 | 1.27586 |      29 |  0.0025392 |    1.03142 |   0.162575 |
+-----------+-----------+-------+-------+----------------+-----------+----------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.0,0.034483,0.0,0.0,0.103448,0.0,0.0,0.0,0.0,...,0.068966,0.034483,0.068966,0.034483,0.137931,0.172414,0.0,0.137931,0.103448,0.0


In [399]:
d_data_donga['TongTanSuatNhietDoCucDoan_ChuanHoa'] =  pd.DataFrame(chuan_hoa_min_max(d_data_donga['TongTanSuatNhietDoCucDoan'].iloc[0], 0.0, 0.142), columns=d_data_donga['TongTanSuatNhietDoCucDoan'].columns)
in_bang_thong_ke_mo_ta(d_data_donga['TongTanSuatNhietDoCucDoan_ChuanHoa'], row_index=0)
d_data_donga['TongTanSuatNhietDoCucDoan_ChuanHoa']

+----------+----------+-------+-------+----------------+----------+---------+---------+---------+------------+------------+------------+
|     mean |      std |   min |   25% |   50% (median) |      75% |     max |     sum |   count |   variance |   skewness |   kurtosis |
+==========+==========+=======+=======+================+==========+=========+=========+=========+============+============+============+
| 0.309826 | 0.354863 |     0 |     0 |       0.242836 | 0.485673 | 1.21418 | 8.98494 |      29 |   0.125927 |    1.03142 |   0.162575 |
+----------+----------+-------+-------+----------------+----------+---------+---------+---------+------------+------------+------------+


,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0.0,0.0,0.242836,0.0,0.0,0.728509,0.0,0.0,0.0,0.0,...,0.485673,0.242836,0.485673,0.242836,0.971345,1.214182,0.0,0.971345,0.728509,0.0


## 3.5* Phân Tách Dữ Liệu Khu Vực

### 3.5.1 Phân tích tương quan

#### 3.5.1.1 Đông Nam Á

In [400]:
corr_dna = d_data_dna['TongDienTichRung_ChuanHoa'].iloc[0].values.reshape(-1, 1)
corr_dna = np.hstack((corr_dna, d_data_dna['TrungBinhNhietDo_ChuanHoa'].iloc[0].values.reshape(-1, 1)))
corr_dna = np.hstack((corr_dna, d_data_dna['TongNongDoCarbon_ChuanHoa'].iloc[0].values.reshape(-1, 1)))
corr_dna = np.hstack((corr_dna, d_data_dna['TongTanSuatThienTai_ChuanHoa'].iloc[0].values.reshape(-1, 1)))
d_data_dna['TuongQuanDNA'] =  pd.DataFrame(corr_dna, columns=['DienTichRung', 'NhietDo', 'NongDoCO2', 'TanSuatThienTai'])
d_data_dna['TuongQuanDNA'].head(5)


,DienTichRung,NhietDo,NongDoCO2,TanSuatThienTai
0,0.816183,0.191816,0.280857,0.289655
1,0.772910,0.156553,0.314573,0.331034
2,0.729637,0.248433,0.347896,0.524138
3,0.686364,0.292898,0.380891,0.386207
4,0.643091,0.164602,0.413632,0.331034


In [401]:
fig = px.line(d_data_dna['TuongQuanDNA'], x=l_timeline, y=d_data_dna['TuongQuanDNA'].columns, title='Biểu đồ đường scatter giữa các cột')
fig.show()

In [402]:
ve_bieu_do_tuong_quan(d_data_dna['TuongQuanDNA'])

**Nhận Xét**

#### 3.5.1.2 Đông Á

In [403]:
corr_donga = d_data_donga['TongDienTichRung_ChuanHoa'].iloc[0].values.reshape(-1, 1)
corr_donga = np.hstack((corr_donga, d_data_donga['TrungBinhNhietDo_ChuanHoa'].iloc[0].values.reshape(-1, 1)))
corr_donga = np.hstack((corr_donga, d_data_donga['TongNongDoCarbon'].iloc[0].values.reshape(-1, 1)))
corr_donga = np.hstack((corr_donga, d_data_donga['TongTanSuatThienTai_ChuanHoa'].iloc[0].values.reshape(-1, 1)))
d_data_donga['TuongQuanDONGA'] =  pd.DataFrame(corr_donga, columns=['DienTichRung', 'NhietDo', 'NongDoCO2', 'TanSuatThienTai'])
d_data_donga['TuongQuanDONGA'].head(5)

,DienTichRung,NhietDo,NongDoCO2,TanSuatThienTai
0,0.020407,0.263158,0.181223,0.127910
1,0.049623,0.111228,0.184130,0.304745
2,0.078839,0.607719,0.187037,0.127910
3,0.108055,0.335439,0.189944,0.068966
4,0.137270,0.036608,0.192851,0.216328


In [404]:
fig = px.line(d_data_donga['TuongQuanDONGA'], x=l_timeline, y=d_data_donga['TuongQuanDONGA'].columns, title='Biểu đồ đường scatter giữa các cột')
fig.show()

In [405]:
ve_bieu_do_tuong_quan(d_data_donga['TuongQuanDONGA'])

# 4 Phân tích tính chất khí hậu Đông Nam Á và Đông Á

## 4.1 Phân tích xu hướng thay đổi

### 4.1.1 Sự thay đổi của nhiệt độ

#### 4.1.1.1 Đông Nam Á

In [406]:
# PCA
d_data_dna['NhietDo_PCA'] = tao_du_lieu_pca(d_data_dna['NhietDo'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['NhietDo_PCA'])

PCA Weight:
ISO3       BRN       IDN       KHM       LAO       MMR       MYS       PHL  \
PC1   0.270263  0.242192  0.281825  0.387584  0.337656  0.279370  0.263291   
PC2   0.144575  0.165741 -0.091685 -0.390439 -0.154772  0.187808  0.135988   

ISO3       SGP       THA       TLS       VNM  
PC1   0.267366  0.357972  0.238168  0.347737  
PC2   0.661167 -0.168290  0.323191 -0.378345  


**Nhận xét:**

**Đa dạng và phân tán:**

- Sự biến đổi nhiệt độ trong khu vực Đông Nam Á được phản ánh qua sự đa dạng và phân tán của các trọng số trên cả PC1 và PC2.

- Các trọng số dương cao của nhiều quốc gia trên cả hai thành phần chính ngụ ý rằng sự biến đổi nhiệt độ không chỉ đóng góp vào một hướng chính mà còn có tính chất đặc biệt hoặc đối lập.

- Ví dụ: Lào (0.387584), Myanmar (0.337656), và Việt Nam (0.347737) đóng góp nhiều nhất vào thành phần chính PC1.
Các trọng số âm cũng cho thấy sự đa dạng trong biến đổi nhiệt độ, đặc biệt trên PC2.

- Ví dụ: Singapore (0.661167) có trọng số dương cao nhất trên PC2, trong khi Thailand (-0.168290) và Vietnam (-0.378345) có trọng số âm, ngụ ý rằng sự biến đổi nhiệt độ của chúng có thể có tính chất đối lập.

**Sự đóng góp của các quốc gia:**

- Trọng số cao của một số quốc gia ngụ ý về mức độ đóng góp của chúng vào sự biến đổi nhiệt độ chung của khu vực.

- Các quốc gia có trọng số dương cao nhất thường là những quốc gia có sự biến đổi nhiệt độ lớn hoặc có xu hướng tương đồng.

- Sự đối lập giữa trọng số của một số quốc gia trên cùng một thành phần chính có thể phản ánh sự đa dạng và phân biệt về biến đổi nhiệt độ giữa các khu vực.
Ví dụ: Trọng số thấp hoặc gần với không của Cambodia (-0.091685) và Laos (-0.154772) ngụ ý rằng sự biến đổi nhiệt độ của những quốc gia này không có ảnh hưởng lớn đến thành phần chính thứ hai của dữ liệu.

**Tóm lại:**
Tính chất của khí hậu trong khu vực Đông Nam Á là đa dạng và phân tán, với sự biến đổi nhiệt độ không chỉ phản ánh sự thay đổi cục bộ mà còn có thể phản ánh các yếu tố toàn cầu và đặc biệt của từng quốc gia. Sự đối lập và không đồng nhất giữa các quốc gia trong khu vực ngụ ý rằng khí hậu của mỗi quốc gia có đặc điểm riêng và có thể phản ánh những ảnh hưởng và yếu tố nội địa khác nhau. Điều này cực kỳ quan trọng khi xem xét các chiến lược phản ứng và ứng phó với biến đổi khí hậu trong khu vực này.

#### 4.1.1.2 Đông Á

In [407]:
# PCA
d_data_donga['NhietDo_PCA'] = tao_du_lieu_pca(d_data_donga['NhietDo'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['NhietDo_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1  -0.333748 -0.301259 -0.413443 -0.640520 -0.465461
PC2   0.096514  0.514033  0.405910 -0.715696  0.222421


**Nhận xét:**

**Tính chất đa dạng và phân tán:**

Sự biến đổi nhiệt dộ trong khu vực Đông Á phản ánh sự đa dạng và phân tán, được thể hiện qua các trọng số trên cả PC1 và PC2.

Các trọng số có giá trị tuyệt đối lớn và đối lập nhau, ngụ ý rằng sự thay đổi nhiệt độ không đồng nhất và phản ánh nhiều yếu tố khác nhau.

**Đóng góp của các quốc gia:**
Tất cả các quốc gia Đông Á có trọng số âm cao trên PC1, đặc biệt là MNG có trọng số âm cao nhất trên cả  PC1 và PC2 (trọng sô lầ lượt là -0.64 à -.71) ngụ ý rằng sự thay đổi khí hậu của chúng đóng góp nhiều vào thành phần chính này.

**Sự đối lập và đa dạng:**

Sự đối lập giữa các trọng số của các quốc gia (ví dụ: CNN, JPN, KỎ và PRK ) có thể phản ánh sự đa dạng về tác động và yếu tố gây ra sự biến đổi khí hậu.


**Tóm Lại: ** Tính chất của khí hậu trong khu vực Đông Á là đa dạng và phân tán, với sự biến đổi nhiệt độ không chỉ phản ánh sự thay đổi cục bộ mà còn có thể phản ánh các yếu tố toàn cầu và đặc biệt của từng quốc gia. Sự đối lập và không đồng nhất giữa các quốc gia trong khu vực ngụ ý rằng khí hậu của mỗi quốc gia có đặc điểm riêng và có thể phản ánh những ảnh hưởng và yếu tố nội địa khác nhau. Điều này cực kỳ quan trọng khi xem xét các chiến lược phản ứng và ứng phó với biến đổi khí hậu trong khu vực này.

### 4.1.2 Sự thay đổi của mực nước biển

#### 4.1.2.1 Đông Nam Á

In [408]:
# PCA
d_data_dna['MucNuoc_PCA'] = tao_du_lieu_pca(d_exdata['MucNuoc'], l_timeline, vung_bien_tiep_giap_dong_nam_a)
phantich_pca(d_data_dna['MucNuoc_PCA'])

PCA Weight:
ISO3  Andaman Sea  Bay Bengal  Indonesian  South China
PC1     -0.501709   -0.372011   -0.645917    -0.438961
PC2      0.463615    0.586596   -0.661935    -0.053000


**Nhận xét:**

**Tính chất đa dạng và phân tán:**

Sự biến đổi mực nước biển trong khu vực Đông Nam Á phản ánh sự đa dạng và phân tán, được thể hiện qua các trọng số trên cả PC1 và PC2.

Các trọng số có giá trị tuyệt đối lớn và đối lập nhau, ngụ ý rằng sự thay đổi mực nước biển của các vùng biển này không đồng nhất và phản ánh nhiều yếu tố khác nhau.

**Đóng góp của các vùng biển:**

Các vùng biển như Andaman Sea và Bay Bengal có trọng số âm cao trên PC1, ngụ ý rằng sự thay đổi mực nước biển của chúng đóng góp nhiều vào thành phần chính này.

Indonesian cũng có trọng số âm cao trên cả PC1 và PC2, cho thấy sự biến đổi mực nước biển ở đây có ảnh hưởng đến cả hai thành phần chính.

**Sự đối lập và đa dạng:**

Sự đối lập giữa các trọng số của các vùng biển (ví dụ: Andaman Sea và Bay Bengal trên PC1) có thể phản ánh sự đa dạng về tác động và yếu tố gây ra sự biến đổi mực nước biển.

Trọng số đối lập trên PC2 (ví dụ: Bay Bengal và South China) cũng cho thấy sự đa dạng trong sự thay đổi mực nước biển giữa các vùng biển.

**Tóm Lại:**
Tính chất của sự thay đổi mực nước biển trong khu vực Đông Nam Á là đa dạng và phân tán, với sự đóng góp đặc biệt từ các vùng biển như Andaman Sea, Bay Bengal và Indonesian. Sự biến đổi này có thể phản ánh sự ảnh hưởng của các yếu tố toàn cầu và địa phương, và có thể có ảnh hưởng lớn đến môi trường biển và nguồn lợi sinh vật biển trong khu vực. Điều này cần được quan sát và nghiên cứu kỹ lưỡng để đưa ra các biện pháp ứng phó và bảo vệ hiệu quả.







#### 4.1.2.2 Đông Á

In [409]:
# PCA
d_data_donga['MucNuoc_PCA'] = tao_du_lieu_pca(d_exdata['MucNuoc'], l_timeline, vung_bien_tiep_giap_dong_a)
phantich_pca(d_data_donga['MucNuoc_PCA'])

PCA Weight:
ISO3  Sea Japan  South China  Yellow Sea
PC1   -0.545833    -0.614634   -0.569466
PC2   -0.382776     0.787488   -0.483058


**Nhận xét:**

**Tính chất đa dạng và phân tán:**

Sự biến đổi mực nước biển trong khu vực Đông Á phản ánh sự đa dạng và phân tán, được thể hiện qua các trọng số trên cả PC1 và PC2.

Các trọng số có giá trị tuyệt đối lớn và đối lập nhau, ngụ ý rằng sự thay đổi mực nước biển của các vùng biển này không đồng nhất và phản ánh nhiều yếu tố khác nhau.

**Đóng góp của các vùng biển:**

Tát cả các vùng biển đều có trọng số âm cao trên PC1, ngụ ý rằng sự thay đổi mực nước biển của chúng đóng góp nhiều vào thành phần chính này.

South China có trọng số dương rất cao trên PC2, cho tháy sự thay đổi mực nước biển đóng góp nhiều vào thành phần này.

**Sự đối lập và đa dạng:**

Sự đối lập giữa các trọng số của vùng biển South China có thể phản ánh sự đa dạng về tác động và yếu tố gây ra sự biến đổi mực nước biển.


**Tóm Lại**: Tính chất của sự thay đổi mực nước biển trong khu vực Đông Á là đa dạng và phân tán, với sự đóng góp đặc biệt từ vùng biển South China. Sự biến đổi này có thể phản ánh sự ảnh hưởng của các yếu tố toàn cầu và địa phương, và có thể có ảnh hưởng lớn đến môi trường biển và nguồn lợi sinh vật biển trong khu vực. Điều này cần được quan sát và nghiên cứu kỹ lưỡng để đưa ra các biện pháp ứng phó và bảo vệ hiệu quả.

### 4.1.3 Sự thay đổi của diện tích rừng

#### 4.1.3.1 Đông Nam Á

In [410]:
# PCA
d_data_dna['DienTichRung_PCA'] = tao_du_lieu_pca(d_data_dna['DienTichRung'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['DienTichRung_PCA'])

PCA Weight:
ISO3       BRN       IDN       KHM       LAO       MMR       MYS       PHL  \
PC1   0.001425  0.873026  0.112890  0.050929  0.417143  0.054167  0.028608   
PC2   0.001987 -0.460719  0.422789  0.108033  0.671553 -0.009445  0.004091   

ISO3       SGP       THA       TLS       VNM  
PC1  -0.000029 -0.044377  0.001661 -0.206769  
PC2   0.000402 -0.261814  0.003452 -0.278682  


**Nhận xét:**

**Tính chất biến đổi của diện tích rừng:**

Trong khu vực này, diện tích rừng biến đổi đáng kể, được phản ánh qua các trọng số trên cả PC1 và PC2.

Các quốc gia có trọng số cao nhất, như Indonesia trên PC1 (0.873026) và Myanmar trên PC2 (0.671553), ngụ ý rằng diện tích rừng của họ có ảnh hưởng lớn đến các thành phần chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Indonesia đóng góp nhiều nhất vào thành phần chính PC1, ngụ ý rằng diện tích rừng của nước này có mức độ ảnh hưởng lớn đến xu hướng chung của dữ liệu.

Các quốc gia khác như Myanmar, Cambodia và Vietnam cũng có trọng số cao, đặc biệt là trên PC2, ngụ ý rằng diện tích rừng của họ cũng đóng góp quan trọng vào các thành phần chính của dữ liệu.

**Sự đối lập và đa dạng:**

Sự đối lập giữa các trọng số của các quốc gia (ví dụ: Indonesia và Vietnam trên PC1) có thể phản ánh sự đa dạng về diện tích rừng và biến đổi của chúng.

Trọng số đối lập trên PC2 (ví dụ: Myanmar và Thailand) cũng cho thấy sự đa dạng trong diện tích rừng giữa các quốc gia.

**Nhận xét tổng quát:**
Diện tích rừng trong khu vực Đông Nam Á đa dạng và phong phú, với sự đóng góp lớn từ các quốc gia như Indonesia, Myanmar và Vietnam. Sự biến đổi này phản ánh tình hình thực tế của nguồn tài nguyên rừng trong khu vực và có thể có ảnh hưởng đến quản lý và bảo vệ rừng trong tương lai. Điều này cần được xem xét kỹ lưỡng khi phát triển các chiến lược và chính sách liên quan đến bảo vệ môi trường và phát triển bền vững.

#### 4.1.3.2 Đông Á

In [411]:
# PCA
d_data_donga['DienTichRung_PCA'] = tao_du_lieu_pca(d_data_donga['DienTichRung'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['DienTichRung_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1  -0.999913 -0.000857  0.004136  0.002842  0.012153
PC2   0.011326  0.316056 -0.052863  0.183348  0.929284


**Tính chất biến đổi của diện tích rừng:**

Trong khu vực này, diện tích rừng biến đổi đáng kể, được phản ánh qua các trọng số trên cả PC1 và PC2.

Các quốc gia có trọng số cao nhất, như China trên PC1 (-0.999913) và North Korea trên PC2 (0.929284), ngụ ý rằng diện tích rừng của họ có ảnh hưởng lớn đến các thành phần chính của dữ liệu.

**Đóng góp của từng quốc gia: **

China đóng góp nhiều nhất vào thành phần chính PC1, ngụ ý rằng diện tích rừng của nước này có mức độ ảnh hưởng lớn đến xu hướng chung của dữ liệu.

North Korea đóng góp nhiều nhất vào thành phần chính , ngụ ý rằng diện tích rừng của nước này có mức độ ảnh hưởng lớn đến xu hướng chung của dữ liệu.

Japan cũng có trọng số cao trên PC2, ngụ ý rằng diện tích rừng của họ cũng đóng góp quan trọng vào các thành phần chính của dữ liệu.

**Sự đối lập và đa dạng:**

Sự đối lập giữa các trọng số của các quốc gia (ví dụ: Trung Quốc và Hàn Quốc PC1) có thể phản ánh sự đa dạng về diện tích rừng và biến đổi của chúng.

Trọng số đối lập trên PC2 (ví dụ: Hàn Quốc và North Korea) cũng cho thấy sự đa dạng trong diện tích rừng giữa các quốc gia.

**Nhận xét tổng quát:** Diện tích rừng trong khu vực Đông Á đa dạng và phong phú, với sự đóng góp lớn từ các quốc gia như China và . Sự biến đổi này phản ánh tình hình thực tế của nguồn tài nguyên rừng trong khu vực và có thể có ảnh hưởng đến quản lý và bảo vệ rừng trong tương lai. Điều này cần được xem xét kỹ lưỡng khi phát triển các chiến lược và chính sách liên quan đến bảo vệ môi trường và phát triển bền vững.

### 4.1.4 Sự thay đổi của diện tích đất

#### 4.1.4.1 Đông Nam Á

In [412]:
# PCA
d_data_dna['DienTichDat_PCA'] = tao_du_lieu_pca(d_data_dna['DienTichDat'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['DienTichDat_PCA'])

PCA Weight:
ISO3  BRN       IDN           KHM  LAO       MMR  MYS           PHL       SGP  \
PC1  -0.0 -0.981225  0.000000e+00  0.0  0.005106  0.0  2.736911e-48 -0.000427   
PC2   0.0 -0.188245 -1.110223e-16 -0.0  0.191863 -0.0  1.368456e-48 -0.010252   

ISO3  THA  TLS       VNM  
PC1   0.0  0.0  0.192800  
PC2  -0.0 -0.0 -0.963144  


**Nhận xét:**

**Tính chất biến đổi của diện tích đất:**

Trọng số PCA cho thấy sự biến đổi diện tích đất của các quốc gia trong khu vực không đồng nhất, có thể đóng góp vào các thành phần chính khác nhau của dữ liệu.
Có sự đối lập giữa các trọng số, như trên PC1, trọng số của Indonesia có giá trị âm rất cao (-0.981225) trong khi trọng số của Myanmar có giá trị dương nhỏ (0.005106), ngụ ý rằng sự biến đổi diện tích đất của hai quốc gia này có hướng phản đối nhau.

**Đóng góp của từng quốc gia:**

Trọng số cao nhất thường đại diện cho quốc gia có ảnh hưởng lớn nhất đến thành phần chính tương ứng. Trong trường hợp này, Indonesia có trọng số âm cao nhất trên PC1, ngụ ý rằng diện tích đất của nước này có ảnh hưởng mạnh mẽ đến thành phần chính này.

Việc trọng số của Việt Nam có giá trị dương lớn nhất trên PC2 (0.192800) cho thấy diện tích đất của nước này có thể đóng góp nhiều vào thành phần chính thứ hai.

**Nhận xét tổng quát:**
Diện tích đất của các quốc gia trong khu vực có sự biến đổi đa dạng, với mức độ ảnh hưởng khác nhau đến các thành phần chính của dữ liệu. Sự đối lập trong trọng số giữa các quốc gia có thể phản ánh sự đa dạng và phức tạp của tình hình diện tích đất trong khu vực. Điều này cần được xem xét kỹ lưỡng khi phát triển các chiến lược và chính sách về sử dụng đất và quản lý tài nguyên đất đất.

#### 4.1.4.2 Đông Á

In [413]:
# PCA
d_data_donga['DienTichDat_PCA'] = tao_du_lieu_pca(d_data_donga['DienTichDat'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['DienTichDat_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG  PRK
PC1   0.002227  0.003727 -0.077813 -0.996959  0.0
PC2   0.008223  0.013092 -0.996844  0.077871 -0.0


**Nhận xét:**

**Tính chất biến đổi của diện tích đất:**

Trọng số PCA cho thấy sự biến đổi diện tích đất của các quốc gia trong khu vực không đồng nhất, có thể đóng góp vào các thành phần chính khác nhau của dữ liệu. Có sự đối lập giữa các trọng số, như trên PC1, trọng số của Mongolia có giá trị âm rất cao (-0.996959) trong khi trọng số của China có giá trị dương nhỏ (0.002227), ngụ ý rằng sự biến đổi diện tích đất của hai quốc gia này có hướng phản đối nhau.

**Đóng góp của từng quốc gia:**

Trọng số cao nhất thường đại diện cho quốc gia có ảnh hưởng lớn nhất đến thành phần chính tương ứng. Trong trường hợp này, Mônglia có trọng số âm cao nhất trên PC1, và Korea có trọng số âm cao nhất trên PC2, ngụ ý rằng diện tích đất của các nước này có ảnh hưởng mạnh mẽ đến thành phần chính này


**Nhận xét tổng quát:** Diện tích đất của các quốc gia trong khu vực có sự biến đổi đa dạng, với mức độ ảnh hưởng khác nhau đến các thành phần chính của dữ liệu. Sự đối lập trong trọng số giữa các quốc gia có thể phản ánh sự đa dạng và phức tạp của tình hình diện tích đất trong khu vực. Điều này cần được xem xét kỹ lưỡng khi phát triển các chiến lược và chính sách về sử dụng đất và quản lý tài nguyên đất đất.

### 4.1.5 Sự thay đổi của nồng độ carbon

#### 4.1.5.1 Đông Nam Á

In [414]:
# PCA
d_data_dna['NongDoCarbon_PCA'] = tao_du_lieu_pca(d_data_dna['NongDoCarbon'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['NongDoCarbon_PCA'])

PCA Weight:
ISO3       BRN       IDN           KHM           LAO       MMR       MYS  \
PC1  -0.000001  0.241021  4.791233e-08 -2.574196e-08 -0.058590 -0.007958   
PC2   0.000002 -0.336541 -5.460863e-08  7.228751e-09 -0.004017  0.602821   

ISO3       PHL       SGP       THA           TLS       VNM  
PC1   0.341712  0.883622  0.025375 -4.770876e-07 -0.200531  
PC2  -0.639574  0.319157 -0.004461 -5.338198e-07 -0.111330  


**Nhận xét:**

**Tính chất biến đổi của nồng độ carbon trên diện tích:**

Trọng số PCA cho thấy sự biến đổi nồng độ carbon trên diện tích của các quốc gia không đồng nhất. Có sự đa dạng trong mức độ ảnh hưởng của từng quốc gia đối với các thành phần chính của dữ liệu.

Một số quốc gia có trọng số gần bằng không hoặc rất nhỏ, như Brunei (BRN) và Timor-Leste (TLS), ngụ ý rằng nồng độ carbon trên diện tích của họ không có ảnh hưởng lớn đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Singapore (SGP) có trọng số cao nhất trên PC1, với giá trị là 0.883622. Điều này ngụ ý rằng nồng độ carbon trên diện tích của Singapore có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể liên quan đến hoạt động công nghiệp và đô thị hóa.

Malaysia (MYS) có trọng số cao nhất trên PC2, với giá trị là 0.602821. Điều này cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể phản ánh tác động của các hoạt động nông nghiệp và lâm nghiệp.

**So sánh giữa các quốc gia:**

Indonesia (IDN) có trọng số dương mạnh trên cả PC1 và PC2, ngụ ý rằng nước này có ảnh hưởng đến cả hai thành phần chính của dữ liệu. Điều này có thể liên quan đến diện tích đất rộng lớn và hoạt động công nghiệp phát triển.

Philippines (PHL) có trọng số dương cao trên PC1 và trọng số âm khá thấp trên PC2, chỉ ra sự đóng góp lớn vào thành phần chính thứ nhất và sự đối lập đối với thành phần chính thứ hai.

**Nhận xét tổng quát:**
Dữ liệu nồng độ carbon trên diện tích của các quốc gia trong khu vực Đông Nam Á có sự biến đổi đa dạng, với mức độ ảnh hưởng khác nhau đến các thành phần chính của dữ liệu. Sự đóng góp lớn từ các quốc gia như Singapore, Indonesia và Malaysia cho thấy tầm quan trọng của việc quản lý nồng độ carbon trong môi trường tự nhiên của khu vực. Điều này cung cấp thông tin quan trọng cho việc đưa ra các biện pháp ứng phó và chính sách về biến đổi khí hậu và bảo vệ môi trường trong khu vực Đông Nam Á.

#### 4.1.5.2 Đông Á

In [415]:
# PCA
d_data_donga['NongDoCarbon_PCA'] = tao_du_lieu_pca(d_data_donga['NongDoCarbon'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['NongDoCarbon_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR           MNG       PRK
PC1   0.093173  0.359907  0.928228 -1.612140e-08  0.013348
PC2   0.725827 -0.659939  0.183915  8.827255e-07 -0.061892


**Nhận xét:**

**Tính chất biến đổi của nồng độ carbon trên diện tích:**

Trọng số PCA cho thấy sự biến đổi nồng độ carbon trên diện tích của các quốc gia không đồng nhất. Có sự đa dạng trong mức độ ảnh hưởng của từng quốc gia đối với các thành phần chính của dữ liệu.

Một số quốc gia có trọng số gần bằng không hoặc rất nhỏ, như Mongolia và North Korea, ngụ ý rằng nồng độ carbon trên diện tích của họ không có ảnh hưởng lớn đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Korea có trọng số cao nhất trên PC1, với giá trị là 0.928228. Điều này ngụ ý rằng nồng độ carbon trên diện tích của Korea có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể liên quan đến hoạt động công nghiệp và đô thị hóa.

China có trọng số cao nhất trên PC2, với giá trị là  0.725827. Điều này cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể phản ánh tác động của các hoạt động nông nghiệp và lâm nghiệp. Japan có giá trị âm cao nhất trên PC2 là -0.659939.

**So sánh giữa các quốc gia:**

China, Korea có trọng số dương cao trên PC này và trọng số âm khá thấp trên PC khác, chỉ ra sự đóng góp lớn vào thành phần chính thứ nhất và sự đối lập đối với thành phần chính thứ hai.

**Nhận xét tổng quát:** Dữ liệu nồng độ carbon trên diện tích của các quốc gia trong khu vực Đông Á có sự biến đổi đa dạng, với mức độ ảnh hưởng khác nhau đến các thành phần chính của dữ liệu. Sự đóng góp lớn từ các quốc gia như China, Korea, Japan cho thấy tầm quan trọng của việc quản lý nồng độ carbon trong môi trường tự nhiên của khu vực. Điều này cung cấp thông tin quan trọng cho việc đưa ra các biện pháp ứng phó và chính sách về biến đổi khí hậu và bảo vệ môi trường trong khu vực Đông  Á.

### 4.1.6 Sự thay đổi của tần suất tổng số thiên tai

#### 4.1.6.1 Đông Nam Á

In [416]:
# PCA
d_data_dna['TanSuatThienTai_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatThienTai'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatThienTai_PCA'])

PCA Weight:
ISO3       BRN       IDN       KHM       LAO       MMR       MYS       PHL  \
PC1  -0.008142  0.272826  0.027636  0.027305  0.039073 -0.040869  0.959157   
PC2  -0.007511  0.871246  0.032940  0.011864  0.098665  0.160292 -0.257084   

ISO3           SGP       THA       TLS       VNM  
PC1  -2.117582e-22  0.013048  0.006302  0.024532  
PC2  -0.000000e+00  0.095010  0.050792  0.355697  


**Nhận xét:**

**Tính chất biến đổi của tần suất thiên tai:**

Trọng số PCA cho thấy sự biến đổi tần suất thiên tai của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Quốc gia có trọng số gần bằng không hoặc rất nhỏ, như Singapore (SGP), ngụ ý rằng tần suất thiên tai không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Philippines (PHL) có trọng số cao nhất trên PC1 (0.959157), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các thiên tai như bão và lũ lụt.

Indonesia (IDN) có trọng số cao nhất trên PC2 (0.871246), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể liên quan đến tần suất và cường độ cao của các trận động đất và núi lửa.

**Nhận xét tổng quát:**
Dữ liệu về tần suất thiên tai của các quốc gia trong khu vực Đông Nam Á cho thấy sự đa dạng và phức tạp của mối liên hệ giữa các yếu tố tự nhiên và nhân tạo. Việc hiểu biết sâu hơn về tần suất và tính chất của các hiện tượng thiên tai có thể giúp cải thiện khả năng ứng phó và giảm thiểu rủi ro cho cộng đồng.

#### 4.1.6.2 Đông Á

In [417]:
# PCA
d_data_donga['TanSuatThienTai_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatThienTai'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatThienTai_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1   0.982215  0.173908 -0.064211 -0.029602 -0.003253
PC2  -0.130510  0.896157  0.413533  0.027477  0.090029


**Nhận xét:**

**Tính chất biến đổi của tần suất thiên tai:**

Trọng số PCA cho thấy sự biến đổi tần suất thiên tai của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Quốc gia có trọng số gần bằng không hoặc rất nhỏ, như Mongolia, ngụ ý rằng tần suất thiên tai không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

China có trọng số cao nhất trên PC1 (0.982215), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các thiên tai như bão và lũ lụt.

Japan có trọng số cao nhất trên PC2 (0.896157), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể liên quan đến tần suất và cường độ cao của các trận động đất và núi lửa.

**Nhận xét tổng quát**: Dữ liệu về tần suất thiên tai của các quốc gia trong khu vực Đông Á cho thấy sự đa dạng và phức tạp của mối liên hệ giữa các yếu tố tự nhiên và nhân tạo. Việc hiểu biết sâu hơn về tần suất và tính chất của các hiện tượng thiên tai có thể giúp cải thiện khả năng ứng phó và giảm thiểu rủi ro cho cộng đồng.

### 4.1.7 Sự thay đổi tần suất lũ lụt

#### 4.1.7.1 Đông Nam Á

In [418]:
# PCA
d_data_dna['TanSuatLuLut_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatLuLut'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatLuLut_PCA'])

PCA Weight:
ISO3  BRN       IDN       KHM       LAO       MMR       MYS       PHL  SGP  \
PC1  -0.0  0.975559 -0.014553  0.004589  0.046381  0.180363 -0.024111  0.0   
PC2   0.0  0.043196  0.038503  0.079054 -0.012425 -0.139682  0.979727 -0.0   

ISO3       THA       TLS       VNM  
PC1   0.049507  0.038897  0.093936  
PC2  -0.012426 -0.031119  0.098734  


**Nhận xét:**

**Tính chất biến đổi của tần suất lũ lụt:**

Trọng số PCA cho thấy sự biến đổi tần suất lũ lụt của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Các quốc gia như Brunei (BRN) và Singapore (SGP) có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất lũ lụt không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Indonesia (IDN) có trọng số cao nhất trên PC1 (0.975559), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các trận lũ lụt.

Philippines (PHL) có trọng số cao nhất trên PC2 (0.979727), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của lũ lụt.

**Nhận xét tổng quát:**
Dữ liệu về tần suất lũ lụt của các quốc gia trong khu vực Đông Nam Á cho thấy sự đa dạng và phức tạp của tình hình môi trường và địa hình trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của lũ lụt có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

#### 4.1.7.2 Đông Á

In [419]:
# PCA
d_data_donga['TanSuatLuLut_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatLuLut'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatLuLut_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1   0.997242  0.029897 -0.013261 -0.058823  0.031298
PC2  -0.030028  0.253925  0.641421 -0.147886  0.708046


**Nhận xét:**

**Tính chất biến đổi của tần suất lũ lụt:**

Trọng số PCA cho thấy sự biến đổi tần suất lũ lụt của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

**Đóng góp của từng quốc gia:**

China  có trọng số cao nhất trên PC1 (0.997242), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các trận lũ lụt.

Mongolia có trọng số cao nhất trên PC2 (0.708046), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của lũ lụt. Korea có trọng số cao chỉ đứng sau Mongolia là 0.641421.

**Nhận xét tổng quát:** Dữ liệu về tần suất lũ lụt của các quốc gia trong khu vực Đông Á cho thấy sự đa dạng và phức tạp của tình hình môi trường và địa hình trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của lũ lụt có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

### 4.1.8 Sự thay đổi tần suất hạn hán

#### 4.1.8.1 Đông Nam Á

In [420]:
# PCA
d_data_dna['TanSuatHanHan_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatHanHan'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatHanHan_PCA'])

PCA Weight:
ISO3  BRN       IDN       KHM       LAO           MMR       MYS       PHL  \
PC1   0.0  0.105790  0.082356  0.234484  0.000000e+00  0.009071  0.343139   
PC2  -0.0  0.143638  0.620583 -0.054193 -5.551115e-17 -0.087183  0.393894   

ISO3  SGP       THA       TLS       VNM  
PC1   0.0  0.683539 -0.064338  0.581250  
PC2   0.0 -0.499350  0.301449  0.297209  


**Nhận xét:**

**Tính chất biến đổi của tần suất hạn hán:**

Trọng số PCA cho thấy sự biến đổi tần suất hạn hán của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Các quốc gia như Brunei (BRN) và Singapore (SGP) có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất hạn hán không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Thailand (THA) có trọng số cao nhất trên PC1 (0.683539), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các hiện tượng hạn hán.

Philippines (PHL) có trọng số cao nhất trên PC2 (0.393894), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của hạn hán.

**Nhận xét tổng quát:**
Dữ liệu về tần suất hạn hán của các quốc gia trong khu vực Đông Nam Á cho thấy sự đa dạng và phức tạp của tình hình khí hậu và môi trường trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của hạn hán có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

#### 4.1.8.2 Đông Á

In [421]:
# PCA
d_data_donga['TanSuatHanHan_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatHanHan'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatHanHan_PCA'])

PCA Weight:
ISO3       CHN           JPN       KOR       MNG       PRK
PC1   0.991206 -0.000000e+00  0.039647  0.080314 -0.097407
PC2   0.096648  1.110223e-16 -0.019659  0.023487  0.994847


**Nhận xét:**

**Tính chất biến đổi của tần suất hạn hán:**

Trọng số PCA cho thấy sự biến đổi tần suất hạn hán của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Japan có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất hạn hán không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

China có trọng số cao nhất trên PC1 (0.991206), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các hiện tượng hạn hán.

Mongolia có trọng số cao nhất trên PC2 (0.994847), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của hạn hán.

**Nhận xét tổng quát:** Dữ liệu về tần suất hạn hán của các quốc gia trong khu vực Đông Á cho thấy sự đa dạng và phức tạp của tình hình khí hậu và môi trường trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của hạn hán có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

### 4.1.9 Sự thay đổi tần suất sạt lở

#### 4.1.9.1 Đông Nam Á

In [422]:
# PCA
d_data_dna['TanSuatSatLo_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatSatLo'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatSatLo_PCA'])

PCA Weight:
ISO3  BRN       IDN           KHM  LAO       MMR       MYS       PHL  SGP  \
PC1   0.0  0.964771  0.000000e+00  0.0  0.024231 -0.068322  0.223964  0.0   
PC2  -0.0 -0.210743  5.551115e-17 -0.0 -0.212423  0.089941  0.949244 -0.0   

ISO3       THA  TLS      VNM  
PC1  -0.003134  0.0  0.11744  
PC2   0.031400 -0.0  0.01799  


**Nhận xét:**

**Tính chất biến đổi của tần suất sạt lở:**

Trọng số PCA cho thấy sự biến đổi tần suất sạt lở của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Các quốc gia như Brunei (BRN) và Singapore (SGP) có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất sạt lở không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Indonesia (IDN) có trọng số cao nhất trên PC1 (0.964771), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các sự kiện sạt lở.

Philippines (PHL) có trọng số cao nhất trên PC2 (0.949244), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của sạt lở.

**Nhận xét tổng quát:**
Dữ liệu về tần suất sạt lở của các quốc gia trong khu vực Đông Nam Á cho thấy sự đa dạng và phức tạp của tình hình môi trường và địa hình trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của sạt lở có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.







#### 4.1.9.2 Đông Á

In [423]:
# PCA
d_data_donga['TanSuatSatLo_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatSatLo'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatSatLo_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG  PRK
PC1   0.999155  0.036733  0.004652  0.017819  0.0
PC2  -0.035902  0.960565  0.273051 -0.038324 -0.0


**Nhận xét:**

**Tính chất biến đổi của tần suất sạt lở:**

Trọng số PCA cho thấy sự biến đổi tần suất sạt lở của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Mongolia có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất sạt lở không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

China có trọng số cao nhất trên PC1 (0.999155), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các sự kiện sạt lở.

Japan có trọng số cao nhất trên PC2 (0.960565), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của sạt lở.

**Nhận xét tổng quát:** Dữ liệu về tần suất sạt lở của các quốc gia trong khu vực Đông Á cho thấy sự đa dạng và phức tạp của tình hình môi trường và địa hình trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của sạt lở có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

### 4.1.10 Sự thay đổi tần suất Bão

#### 4.1.10.1 Đông Nam Á

In [424]:
# PCA
d_data_dna['TanSuatBao_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatBao'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatBao_PCA'])

PCA Weight:
ISO3  BRN       IDN       KHM       LAO       MMR       MYS       PHL  SGP  \
PC1  -0.0  0.023582  0.028488 -0.012440  0.045688 -0.041938  0.980258  0.0   
PC2   0.0 -0.009957  0.133146  0.127329  0.035709 -0.005207  0.159644  0.0   

ISO3       THA       TLS       VNM  
PC1  -0.099997  0.009654 -0.153731  
PC2   0.129858  0.026193  0.960012  


**Nhận xét:**

**Tính chất biến đổi của tần suất bão:**

Trọng số PCA cho thấy sự biến đổi tần suất bão của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

Các quốc gia như Brunei (BRN) và Singapore (SGP) có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất bão không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu.

**Đóng góp của từng quốc gia:**

Philippines (PHL) có trọng số cao nhất trên PC1 (0.980258), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các trận bão.

Vietnam (VNM) có trọng số cao nhất trên PC2 (0.960012), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của bão.

**Nhận xét tổng quát:**
Dữ liệu về tần suất bão của các quốc gia trong khu vực Đông Nam Á cho thấy sự đa dạng và phức tạp của tình hình khí hậu và môi trường trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của bão có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

#### 4.1.10.2 Đông Á

In [425]:
# PCA
d_data_donga['TanSuatBao_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatBao'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatBao_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1   0.947679  0.312040 -0.056828 -0.012754 -0.033826
PC2  -0.255094  0.858556  0.443761  0.022734  0.019190


### 4.1.11 Sự thay đổi tần suất Nhiệt Độ Cực Đoan

**Tính chất biến đổi của tần suất bão:**

Trọng số PCA cho thấy sự biến đổi tần suất bão của các quốc gia không đồng nhất, với mức độ ảnh hưởng khác nhau đến thành phần chính của dữ liệu.

**Đóng góp của từng quốc gia:**

China có trọng số cao nhất trên PC1 (0.947679), ngụ ý rằng nước này có ảnh hưởng mạnh mẽ đến thành phần chính này, có thể do tần suất cao của các trận bão.

Japan có trọng số cao nhất trên PC2 (0.858556), cho thấy nước này có ảnh hưởng lớn đến thành phần chính thứ hai, có thể do tần suất và cường độ cao của bão.

**Nhận xét tổng quát:** Dữ liệu về tần suất bão của các quốc gia trong khu vực Đông Á cho thấy sự đa dạng và phức tạp của tình hình khí hậu và môi trường trong khu vực này. Việc hiểu biết sâu hơn về tần suất và tính chất của bão có thể giúp cải thiện khả năng dự báo và ứng phó trong tình huống khẩn cấp và giảm thiểu thiệt hại cho cộng đồng.

#### 4.1.11.1 Đông Nam Á

In [426]:
# PCA
d_data_dna['TanSuatNhietDoCucDoan_PCA'] = tao_du_lieu_pca(d_data_dna['TanSuatNhietDoCucDoan'], l_timeline, ma_iso_dna)
phantich_pca(d_data_dna['TanSuatNhietDoCucDoan_PCA'])

PCA Weight:
ISO3  BRN  IDN  KHM  LAO  MMR  MYS  PHL  SGP  THA  TLS  VNM
PC1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
PC2   0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


**Nhận xét:**

**Tính chất biến đổi của tần suất nhiệt độ cực đoan:**

Trọng số PCA chỉ có một giá trị lớn ở PC1, với trọng số của quốc gia Thailand (THA) bằng 1.0.

Các quốc gia khác có trọng số gần bằng không hoặc rất nhỏ, ngụ ý rằng tần suất nhiệt độ cực đoan không có ảnh hưởng đáng kể đến biến đổi chính của dữ liệu ngoại trừ Thailand.

**Nhận xét tổng quát:**
Dữ liệu về tần suất nhiệt độ cực đoan của các quốc gia trong khu vực Đông Nam Á chỉ cho thấy sự ảnh hưởng lớn từ nước Thailand. Có thể dựa vào các yếu tố khác như địa hình, địa vị địa lý và điều kiện hải quan để hiểu rõ hơn về tần suất nhiệt độ cực đoan trong khu vực này.

In [430]:
ve_do_thi_bien_doi_fourier(d_data_dna['TongTanSuatNhietDoCucDoan_ChuanHoa'].iloc[0].tolist(), 1, l_timeline)

Fourier Coefficients:
Frequency: 0.0 Hz, Magnitude: 1.603849238171612
Frequency: 0.034482758620689655 Hz, Magnitude: 1.5663521342583588
Frequency: 0.06896551724137931 Hz, Magnitude: 1.4556141454225457
Frequency: 0.10344827586206896 Hz, Magnitude: 1.2768132569456998
Frequency: 0.13793103448275862 Hz, Magnitude: 1.0383099996500844
Frequency: 0.1724137931034483 Hz, Magnitude: 0.7512565207695109
Frequency: 0.20689655172413793 Hz, Magnitude: 0.42907512193940783
Frequency: 0.24137931034482757 Hz, Magnitude: 0.0868306472901645
Frequency: 0.27586206896551724 Hz, Magnitude: 0.2594739319210341
Frequency: 0.3103448275862069 Hz, Magnitude: 0.5936457984601675
Frequency: 0.3448275862068966 Hz, Magnitude: 0.9000594472532195
Frequency: 0.3793103448275862 Hz, Magnitude: 1.164387316636938
Frequency: 0.41379310344827586 Hz, Magnitude: 1.3742697292182668
Frequency: 0.4482758620689655 Hz, Magnitude: 1.5198928166524501
Frequency: 0.48275862068965514 Hz, Magnitude: 1.5944474052195021
Frequency: -0.4827586206

**Nhận xét:**

**Thành phần DC (Frequency 0 Hz):**

Thành phần này đại diện cho giá trị trung bình của tín hiệu. Với giá trị biên độ 290.0 (cần chia cho số lượng mẫu để có giá trị trung bình thực tế), nó cho thấy rằng giá trị trung bình của tín hiệu trong giai đoạn này là khoảng 10 (290 / 29).

**Tần số 0.034 Hz:**

Tần số này có biên độ 2.0, đại diện cho một dao động với chu kỳ khoảng 1/0.034 = 29.41 năm (gần bằng 29 năm). Điều này có nghĩa là tín hiệu có một dao động với biên độ 2 mỗi chu kỳ khoảng 29 năm.

**Tần số 0.069 Hz:**

Tần số này có biên độ 3.0, đại diện cho một dao động với chu kỳ khoảng 1/0.069 = 14.49 năm (gần bằng 14.5 năm). Điều này có nghĩa là tín hiệu có một dao động với biên độ 3 mỗi chu kỳ khoảng 14.5 năm.

**Kết Luận **: Tín hiệu nhiệt độ có một giá trị trung bình khoảng 10. Các thành phần tần số trong phổ Fourier cho thấy sự biến đổi theo các chu kỳ khác nhau trong tín hiệu: Một dao động với biên độ 2 mỗi chu kỳ khoảng 29 năm. Một dao động với biên độ 3 mỗi chu kỳ khoảng 14.5 năm.

#### 4.1.11.1 Đông Á

In [431]:
# PCA
d_data_donga['TanSuatNhietDoCucDoan_PCA'] = tao_du_lieu_pca(d_data_donga['TanSuatNhietDoCucDoan'], l_timeline, ma_iso_donga)
phantich_pca(d_data_donga['TanSuatNhietDoCucDoan_PCA'])

PCA Weight:
ISO3       CHN       JPN       KOR       MNG       PRK
PC1  -0.128628  0.905568  0.130230  0.379783  0.046966
PC2   0.987895  0.148862  0.023865 -0.025288 -0.026346


#5.Phân tích ảnh hưởng của diện tích rừng lên nhiệt độ, nồng độ CO2 và tần xuất xuất hiện thiên tai ở Việt Nam

In [432]:
nhietdo_vn =  d_data_dna['NhietDo'].loc[d_data_dna['NhietDo']['ISO3'] == 'VNM']
dtrung_vn = d_data_dna['DienTichRung'].loc[d_data_dna['DienTichRung']['ISO3'] == 'VNM']
nongdo_carbon_vn = d_data_dna['NongDoCarbon'].loc[d_data_dna['NongDoCarbon']['ISO3'] == 'VNM']
tsthientai_vn = d_data_dna['TanSuatThienTai'].loc[d_data_dna['TanSuatThienTai']['ISO3'] == 'VNM']

In [433]:
d_data_vn = pd.DataFrame({
    'NhietDo': nhietdo_vn.values[0],
    'DienTichRung': dtrung_vn.values[0],
    'NongDoCarbon': nongdo_carbon_vn.values[0],
    'TanSuatThienTai': tsthientai_vn.values[0]
})
d_data_vn = d_data_vn.drop(0).reset_index(drop=True)

In [434]:
nhietdo_vn

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,VNM,0.047,0.292,0.358,0.369,0.014,0.318,1.307,0.737,0.122,...,0.128,0.764,0.928,0.551,1.18,1.139,1.103,0.78,1.734,1.418


In [435]:
# Chuẩn Hóa Min Max , min là 0.09 max là 1.8
d_data_vn['NhietDo'] =  pd.DataFrame(chuan_hoa_min_max(d_data_vn['NhietDo'].values, 0.009, 1.8)).transpose()

In [436]:
dtrung_vn

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,VNM,9857.588,10098.402,10339.216,10580.03,10820.844,11061.658,11302.472,11543.286,11784.1,...,13522.82,13657.58,13792.34,13927.1,14061.86,14377.68,14415.39,14491.29,14567.19,14643.09


In [437]:
# Chuẩn Hóa Min Max , min là 9856,   max là 14645
d_data_vn['DienTichRung'] =  pd.DataFrame(chuan_hoa_min_max(d_data_vn['DienTichRung'].values, 9856, 14645)).transpose()

In [438]:
nongdo_carbon_vn

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,VNM,0.035519,0.036772,0.037967,0.039107,0.040197,0.041239,0.042237,0.043193,0.04411,...,0.0364,0.036595,0.036787,0.036975,0.03716,0.0369,0.03738,0.03742,0.03727,0.0373


In [439]:
# Chuẩn Hóa Min Max , min là 0.027,   max là 0.045
d_data_vn['NongDoCarbon'] =  pd.DataFrame(chuan_hoa_min_max(d_data_vn['NongDoCarbon'].values, 0.027, 0.045)).transpose()

In [440]:
tsthientai_vn

,ISO3,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,VNM,0.172414,0.137931,0.103448,0.068966,0.206897,0.137931,0.137931,0.172414,0.37931,...,0.172414,0.137931,0.344828,0.103448,0.172414,0.275862,0.310345,0.241379,0.275862,0.37931


In [441]:
# Chuẩn Hóa Min Max , min là 0.054,   max là 0.045
d_data_vn['TanSuatThienTai'] =  pd.DataFrame(chuan_hoa_min_max(d_data_vn['TanSuatThienTai'].values, 0.054, 0.45)).transpose()

In [442]:
l_timeline_vn =  [str(e) for e in range(1992, 2021, 1)]
fig = px.line(d_data_donga['TuongQuanDONGA'], x=l_timeline_vn, y=d_data_donga['TuongQuanDONGA'].columns, title='Biểu đồ đường scatter giữa các cột')
fig.show()

**Nhận xét:**

+ Trong suốt giai đoạn từ năm 1992 đến 2020, diện tích rừng tăng lên một cách đáng kinh ngạc từ ~0.0 đến 0.9.

+ Nhiệt độ và Tuần suất thiên tai dao động mạnh mẽ, sự dao động này có thể là do bị tác động bởi rất nhiều lí do khác nhau.

+ Nhiệt độ có xu hướng tăng mạnh và chạm đỉnh vào năm 2020 ở mức ~2.0

+ Tuần suất thiên tai chho tháy có xu hướng giảm bắt đầu từ năm 2014 đến 2020, đạt mức 0.2 vào cuối giai đoạn

+ Nồng độ CO2 có xu hướng tăng rất nhẹ từ ~0.2 đến 0.3 vào năm 2020.

In [443]:
ve_bieu_do_tuong_quan(d_data_vn)

**Nhận xét:**

+ Diện tích rừng có độ tương quan dương cao nhất với Nhiệt độ ở mức 0.62. cho thấy diện tích rừng và nhiệt độ có mối quan hệ mật thiết.

+ Diện tích rừng cũng có độ tương quan dương cao với Tần suất thiên tai ở mức 0.46. Thiên tai cũng bao gồm các hiện tượng thiên nhiên liên quan đến rừng rậm, vậy nên diện tích rừng cũng đóng góp một phần lớn vào các vụ thiên tai sảy ra.

+ Ngược lại, diện tích rừng có độ tương quan âm với Nồng độ Carbon ở mức -0.4.


# 6*. Xây dựng mô hình Logistic Regression dự đoạn tần suất xuất hiện hạn hán và lũ lụt ở Việt Nam

In [444]:
from sklearn.linear_model import LogisticRegression

In [445]:
index = [str(e) for e in range(1992, 2021)]

In [446]:
nhietdo_vn = d_exdata['NhietDo'].loc[d_exdata['NhietDo']['ISO3'] == 'VNM', index]

In [447]:
def filter_dataframe(df, iso3_column='ISO3', iso3_value='VNM', columns=index):
    return df.loc[df[iso3_column] == iso3_value, columns]

# Áp dụng hàm cho các DataFrame
nhietdo_vn = filter_dataframe(d_exdata['NhietDo'])
tsthientai_vn = filter_dataframe(d_exdata['TongThienTai'])
bao_vn = filter_dataframe(d_exdata['Bao'])
satlo_vn = filter_dataframe(d_exdata['SatLo'])
nhietdocucdoan_vn = filter_dataframe(d_exdata['NhietDoCucDoan'])
hanhan_vn = filter_dataframe(d_exdata['HanHan'])
lulut_vn = filter_dataframe(d_exdata['LuLut'])
chayrung_vn = filter_dataframe(d_exdata['ChayRung'])
rung_vn = filter_dataframe(d_exdata['DienTichRung'])
dat_vn = filter_dataframe(d_exdata['DienTichDat'])
carbon_vn = filter_dataframe(d_exdata['Carbon_DienTichRung'])

In [448]:
# In ra kích thước của từng DataFrame
print("Kích thước của DataFrame tsthientai_vn:", tsthientai_vn.shape)
print("Kích thước của DataFrame bao_vn:", bao_vn.shape)
print("Kích thước của DataFrame satlo_vn:", satlo_vn.shape)
print("Kích thước của DataFrame nhietdocucdoan_vn:", nhietdocucdoan_vn.shape)
print("Kích thước của DataFrame hanhan_vn:", hanhan_vn.shape)
print("Kích thước của DataFrame lulut_vn:", lulut_vn.shape)
print("Kích thước của DataFrame chayrung_vn:", chayrung_vn.shape)


Kích thước của DataFrame tsthientai_vn: (1, 29)
Kích thước của DataFrame bao_vn: (1, 29)
Kích thước của DataFrame satlo_vn: (1, 29)
Kích thước của DataFrame nhietdocucdoan_vn: (1, 29)
Kích thước của DataFrame hanhan_vn: (1, 29)
Kích thước của DataFrame lulut_vn: (1, 29)
Kích thước của DataFrame chayrung_vn: (1, 29)


##6.1 Dự đoán tần suất xuất hiện hạn hán ở Việt Nam

In [449]:
d_data_hanhan_vn = pd.DataFrame({
    'NhietDo': nhietdo_vn.values[0],
    'TanSuatThienTai': tsthientai_vn.values[0],
    'Bao': bao_vn.values[0],
    'SatLo':satlo_vn.values[0],
    'NhietDoCucDoan': nhietdocucdoan_vn.values[0],
    'ChayRung': chayrung_vn.values[0],
    'DienTichRung': rung_vn.values[0],
    'DienTichDat': dat_vn.values[0],
    'Carbon': carbon_vn.values[0],
    'HanHan': hanhan_vn.values[0]
})

index =  [int(e) for e in range(1992, 2021)]
d_data_hanhan_vn.index = index

In [450]:
d_data_hanhan_vn

,NhietDo,TanSuatThienTai,Bao,SatLo,NhietDoCucDoan,ChayRung,DienTichRung,DienTichDat,Carbon,HanHan
1992,0.047,5.0,2.0,1.0,0.0,0.0,9857.588,32549.000,35519.287274,0.0
1993,0.292,4.0,3.0,0.0,0.0,0.0,10098.402,32549.000,36772.382403,0.0
1994,0.358,3.0,1.0,1.0,0.0,0.0,10339.216,32549.000,37967.105049,0.0
1995,0.369,2.0,1.0,0.0,0.0,0.0,10580.030,32549.000,39107.431642,0.0
1996,0.014,6.0,4.0,0.0,0.0,0.0,10820.844,32549.000,40197.012359,0.0
1997,0.318,4.0,3.0,0.0,0.0,0.0,11061.658,32549.000,41239.152395,1.0
1998,1.307,4.0,3.0,0.0,0.0,0.0,11302.472,32549.000,42236.875261,0.0
1999,0.737,5.0,1.0,0.0,0.0,0.0,11543.286,32549.000,43192.978152,1.0
2000,0.122,11.0,7.0,2.0,0.0,0.0,11784.100,31106.000,44110.004158,0.0
2001,0.679,7.0,3.0,0.0,0.0,0.0,11944.496,31109.000,43223.405994,0.0


In [451]:
X_train_hanhan = d_data_hanhan_vn.drop(columns = 'HanHan')
y_train_hanhan = d_data_hanhan_vn['HanHan']

In [452]:
model_hanhan = LogisticRegression()
model_hanhan.fit(X_train_hanhan, y_train_hanhan)

LogisticRegression()

In [453]:
# Dữ liệu mới
hanhan_2023_vn = np.array([1.333, 8.0, 3.0, 0.0, 0.0, 0.0, 14860.309, 331.212, 37300.002937])

hanhan_2023_vn = hanhan_2023_vn.reshape(1, -1)

prediction = model_hanhan.predict(hanhan_2023_vn)
print("Dự đoán của mô hình:", prediction)

Dự đoán của mô hình: [1.]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



## 6.2 Dự đoán tần suất xuất hiện lũ lụt ở Việt Nam

In [454]:
d_data_lulut_vn = pd.DataFrame({
    'NhietDo': nhietdo_vn.values[0],
    'TanSuatThienTai': tsthientai_vn.values[0],
    'Bao': bao_vn.values[0],
    'SatLo':satlo_vn.values[0],
    'NhietDoCucDoan': nhietdocucdoan_vn.values[0],
    'ChayRung': chayrung_vn.values[0],
    'DienTichRung': rung_vn.values[0],
    'DienTichDat': dat_vn.values[0],
    'Carbon': carbon_vn.values[0],
    'LuLut': lulut_vn.values[0]
})

index =  [int(e) for e in range(1992, 2021)]
d_data_lulut_vn.index = index

In [455]:
d_data_lulut_vn

,NhietDo,TanSuatThienTai,Bao,SatLo,NhietDoCucDoan,ChayRung,DienTichRung,DienTichDat,Carbon,LuLut
1992,0.047,5.0,2.0,1.0,0.0,0.0,9857.588,32549.000,35519.287274,2.0
1993,0.292,4.0,3.0,0.0,0.0,0.0,10098.402,32549.000,36772.382403,1.0
1994,0.358,3.0,1.0,1.0,0.0,0.0,10339.216,32549.000,37967.105049,1.0
1995,0.369,2.0,1.0,0.0,0.0,0.0,10580.030,32549.000,39107.431642,1.0
1996,0.014,6.0,4.0,0.0,0.0,0.0,10820.844,32549.000,40197.012359,2.0
1997,0.318,4.0,3.0,0.0,0.0,0.0,11061.658,32549.000,41239.152395,0.0
1998,1.307,4.0,3.0,0.0,0.0,0.0,11302.472,32549.000,42236.875261,1.0
1999,0.737,5.0,1.0,0.0,0.0,0.0,11543.286,32549.000,43192.978152,3.0
2000,0.122,11.0,7.0,2.0,0.0,0.0,11784.100,31106.000,44110.004158,2.0
2001,0.679,7.0,3.0,0.0,0.0,0.0,11944.496,31109.000,43223.405994,4.0


In [456]:
X_train_lulut = d_data_lulut_vn.drop(columns = 'LuLut')
y_train_lulut = d_data_lulut_vn['LuLut']

In [457]:
model_lulut = LogisticRegression()
model_lulut.fit(X_train_lulut, y_train_lulut)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [458]:
# Dữ liệu mới
lulut_2023_vn = np.array([1.333, 8.0, 3.0, 0.0, 0.0, 0.0, 14860.309, 331.212, 37300.002937])

lulut_2023_vn = lulut_2023_vn.reshape(1, -1)

prediction = model_lulut.predict(lulut_2023_vn)
print("Dự đoán của mô hình:", prediction)

Dự đoán của mô hình: [3.]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names

